# Add FPTrans with VISION24 dataset

In [ ]:
! rm /kaggle/working/FPTrans/data/pretrained/vit/B_16-i1k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_384.npz

## Setup VISION24

In [1]:
! pip install sacred
!pip install dropblock
! cp -r /kaggle/input/fptrans/pytorch/default/1/FPTrans /kaggle/working/
! cp -r /kaggle/input/vision24/VISION24 /kaggle/working/FPTrans/data/

%cd /kaggle/working/FPTrans
! rm -rf /kaggle/working/FPTrans/data/VISION24/weights
! rm /kaggle/working/FPTrans/lists/vision24/test_0.pkl
! rm /kaggle/working/FPTrans/lists/vision24/train_0.pkl
! cp /kaggle/input/vision24/test.txt  /kaggle/working/FPTrans/lists/vision24
! cp /kaggle/input/vision24/train.txt  /kaggle/working/FPTrans/lists/vision24


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

In [37]:
!rm -rf /kaggle/working/FPTrans/data/VISION24/weights

In [15]:
!mkdir /kaggle/working/FPTrans/output/7

In [16]:
!cp /kaggle/input/d/wasimatta/newpath/bestckpt.pth /kaggle/working/FPTrans/output/7

In [17]:
!mkdir /kaggle/working/FPTrans/data/pretrained/deit/

In [18]:
!cp /kaggle/input/deit-backbone/deit_base_distilled_patch16_384-d0272ac0.pth /kaggle/working/FPTrans/data/pretrained/deit/

In [19]:
! python tools.py precompute_loss_weights with dataset=VISION24

/kaggle/working/FPTrans/data/VISION24/weights/000149.npz: 100%|█| 1616/1616 [06:


In [20]:
import re
import matplotlib.pyplot as plt

def plot_training_metrics(log_file):
    with open(log_file, 'r') as f:
        lines = f.readlines()

    # Regular expression to match log lines with metrics
    pattern = r'\[(?P<epoch>\d+)/\d+\] \| Tr (?P<train_loss>\d+\.\d+) \| Val (?P<val_loss>\d+\.\d+) \| mIoU \s*(?P<mIoU>\d+\.\d+) \| bIoU \s*(?P<bIoU>\d+\.\d+) \|.*?Catch Rate (?P<catch_rate>\d+\.\d+)'

    train_losses = []
    val_losses = []
    catch_rates = []

    # Parse each line
    for line in lines:
        match = re.search(pattern, line)
        if match:
            train_loss = float(match.group('train_loss'))
            val_loss = float(match.group('val_loss'))
            catch_rate = float(match.group('catch_rate'))
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            catch_rates.append(catch_rate)

    if not train_losses:
        print("No metrics found in the log file.")
        return

    # Generate epochs list
    epochs = range(1, len(train_losses) + 1)

    # Create plots
    plt.figure(figsize=(12, 6))

    # Training Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid(True)

    # Validation Loss Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, val_losses, label='Val Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Validation Loss')
    plt.legend()
    plt.grid(True)

    # Catch Rate Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, catch_rates, label='Catch Rate', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Catch Rate')
    plt.title('Catch Rate')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()


In [ ]:
# Run training and save logs
! python run.py train with split=0 configs/vision24_vit.yml > training_log.txt

# Plot the metrics
plot_training_metrics('training_log.txt')

2025-05-06 22:09:51: I Running command 'train'
2025-05-06 22:09:51: I Started run with ID "10"
2025-05-06 22:09:51: I RUN DIRECTORY: /kaggle/working/FPTrans/output/10
2025-05-06 22:09:51: I Run:run.py train with split=0 configs/vision24_vit.yml
2025-05-06 22:09:51: I Init ==> split 0, shot 1
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
2025-05-06 22:09:51: I      ==> Data loader VISION24 for train
2025-05-06 22:09:51: I      ==> Data loader VISION24 for eval_online
2025-05-06 22:09:51: I      ==> 6400 training samples
2025-05-06 22:09:51: I      ==> 400 eval_online samples
/kaggle/wo

In [20]:
! python run.py train with split=0 configs/vision24_vit.yml

2025-05-04 19:00:11: I Running command 'train'
2025-05-04 19:00:11: I Started run with ID "5"
2025-05-04 19:00:11: I RUN DIRECTORY: /kaggle/working/FPTrans/output/5
2025-05-04 19:00:11: I Run:run.py train with split=0 configs/vision24_vit.yml
2025-05-04 19:00:11: I Init ==> split 0, shot 1
begin
Processing data...
100%|██████████| 1297/1297 [00:44<00:00, 28.91it/s]
Checking image&label list done! There are 1297 images in split 0.
Class 0: 248 images
Class 1: 200 images
Class 2: 96 images
Class 3: 120 images
Class 4: 104 images
Class 5: 0 images
Class 6: 0 images
Class 7: 0 images
Class 8: 0 images
Class 9: 0 images
Class 10: 448 images
Class 11: 81 images
Data list length: 1297
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creati

In [22]:
pip install timm


Note: you may need to restart the kernel to use updated packages.


In [23]:
! python run.py train with split=0 exp_id=7 configs/vision24_vit.yml > training_log.txt

# Plot the metrics
plot_training_metrics('training_log.txt')

2025-05-06 22:07:23: I Running command 'train'
2025-05-06 22:07:24: I Started run with ID "9"
2025-05-06 22:07:24: I RUN DIRECTORY: /kaggle/working/FPTrans/output/9
2025-05-06 22:07:24: I Run:run.py train with split=0 exp_id=7 configs/vision24_vit.yml
2025-05-06 22:07:24: I Init ==> split 0, shot 1
100%|██████████| 1616/1616 [01:17<00:00, 20.79it/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
2025-05-06 22:08:41: I      ==> Data loader VISION24 for train
100%|██████████| 408/408 [00:18<00:00, 21.74it/s]
2025-05-06 22:09:00: I      ==> Data loader VISION24 for eval_online
2025-05-06 

In [ ]:
! python run.py train with split=0 exp_id=7 configs/vision24_vit.yml

2025-05-05 04:56:10: I Running command 'train'
2025-05-05 04:56:11: I Started run with ID "8"
2025-05-05 04:56:11: I RUN DIRECTORY: /kaggle/working/FPTrans/output/8
2025-05-05 04:56:11: I Run:run.py train with split=0 exp_id=7 configs/vision24_vit.yml
2025-05-05 04:56:11: I Init ==> split 0, shot 1
begin
Class 0: 248 images
Class 1: 200 images
Class 2: 96 images
Class 3: 120 images
Class 4: 104 images
Class 5: 0 images
Class 6: 0 images
Class 7: 0 images
Class 8: 0 images
Class 9: 0 images
Class 10: 448 images
Class 11: 81 images
Data list length: 1297
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  

In [ ]:
! python run.py train with split=0 exp_id = 7 configs/vision24_vit.yml

In [ ]:
! python run.py test with configs/vision24_vit.yml exp_id=7 split=0

In [ ]:
! rm /kaggle/working/FPTrans/lists/vision24/test_0.pkl

In [36]:
! mkdir output/7/

In [ ]:
! cp /kaggle/input/bestmodel/bestckpt_10epochs.pth /kaggle/working/FPTrans/output/7/

In [ ]:
!mv /kaggle/working/FPTrans/output/7/bestckpt_10epochs.pth /kaggle/working/FPTrans/output/7/bestckpt.pth

In [ ]:
! python run.py train with split=0 configs/vision24_vit.yml

In [ ]:
! python run.py test with configs/vision24_vit.yml exp_id=73 split=0

In [ ]:
! python run.py train with split=0 configs/vision24_vit.yml 

In [ ]:
! python run.py train with split=0 exp_id=73 configs/vision24_vit.yml 

In [ ]:
! python run.py test with configs/vision24_vit.yml exp_id=73 split=0

In [ ]:
!find -name "*000561*"
!find -name "results"

In [ ]:
! rm -rf results

In [ ]:
! python run.py predict with configs/vision24_vit.yml exp_id=73 split=0

In [ ]:
! rm /kaggle/working/FPTrans/lists/vision24/test_0.pkl
! rm /kaggle/working/FPTrans/lists/vision24/train_0.pkl

In [2]:
%%writefile /kaggle/working/FPTrans/configs/vision24_vit.yml
# %load /kaggle/working/FPTrans/configs/vision24_vit.yml
dataset: VISION24
#backbone: DeiT-B/16-384
backbone: DeiT-B/16-384

vit_depth: 12  # DeiT-B/16-384 has 12 transformer layers
epochs: 15
lr: 0.0001
bs: 8
train_n: 6400
test_n: 400
bg_num: 1
scale_min: 0.6
scale_max: 1.4
pair_lossW: 0.001
height: 384
width: 384
optim: "adamw"
network: fptrans  # Required by load_model()



# Checkpoint loading
#pretrained: true
# Uncomment and set the correct path to your checkpoint
# ckpt_path: /kaggle/working/FPTrans/checkpoints/fptrans_deit384_exp40.pth
strict: false  # Allows partial loading if minor mismatches persist

# p:
# #Screw_front
#   sup_rgb: ["./data/VISION24/JPEGImages/001757.jpg",
           
#            ]
#   sup_msk: ["./data/VISION24/SegmentationClassAug/001757.png",
            
             
#            ]
#   qry_rgb: ["./data/VISION24/JPEGImages/000432.jpg",
#            "./data/VISION24/JPEGImages/000874.jpg",
#             "./data/VISION24/JPEGImages/000114.jpg",
#             "./data/VISION24/JPEGImages/000860.jpg",
        
            
            
            
            
#            ]

# p:
# #Cylinder_Porosity

#   sup_rgb: ["./data/VISION24/JPEGImages/000723.jpg",
           
#            ]
#   sup_msk: ["./data/VISION24/SegmentationClassAug/000723.png",
            
             
#            ]
#   qry_rgb: ["./data/VISION24/JPEGImages/000756.jpg",
#            "./data/VISION24/JPEGImages/000742.jpg",
#             "./data/VISION24/JPEGImages/000772.jpg",
#             "./data/VISION24/JPEGImages/000690.jpg",
        
            
            
            
            
#            ]
#   out: results/



# p:
# #Wood 

#   sup_rgb: ["./data/VISION24/JPEGImages/000377.jpg",
           
#            ]
#   sup_msk: ["./data/VISION24/SegmentationClassAug/000377.png",
            
             
#            ]
#   qry_rgb: ["./data/VISION24/JPEGImages/000543.jpg",
#            "./data/VISION24/JPEGImages/000484.jpg",
#             "./data/VISION24/JPEGImages/000485.jpg",
#             "./data/VISION24/JPEGImages/000471.jpg",        
#            ]
#   out: results/


# p:
# #Cable_thunderbolt
 

#   sup_rgb: ["./data/VISION24/JPEGImages/000175.jpg",
           
#            ]
#   sup_msk: ["./data/VISION24/SegmentationClassAug/000175.png",
            
             
#            ]
#   qry_rgb: ["./data/VISION24/JPEGImages/000015.jpg",
#            "./data/VISION24/JPEGImages/000218.jpg",
#             "./data/VISION24/JPEGImages/000167.jpg",
#             "./data/VISION24/JPEGImages/000253.jpg",        
#            ]
#   out: results/


# p:
# #Cylinder_RCS

 

#   sup_rgb: ["./data/VISION24/JPEGImages/000903.jpg",
           
#            ]
#   sup_msk: ["./data/VISION24/SegmentationClassAug/000903.png",
            
             
#            ]
#   qry_rgb: ["./data/VISION24/JPEGImages/000860.jpg",
#            "./data/VISION24/JPEGImages/000955.jpg",
#             "./data/VISION24/JPEGImages/000926.jpg",
#             "./data/VISION24/JPEGImages/000878.jpg",        
#            ]
#   out: results/


p:
#PCB_spurious_copper


 

  sup_rgb: ["./data/VISION24/JPEGImages/001312.jpg",
           
           ]
  sup_msk: ["./data/VISION24/SegmentationClassAug/001312.png",
            
             
           ]
  qry_rgb: ["./data/VISION24/JPEGImages/001287.jpg",
           "./data/VISION24/JPEGImages/001302.jpg",
            "./data/VISION24/JPEGImages/001356.jpg",
            "./data/VISION24/JPEGImages/001291.jpg",        
           ]
  out: results/

Overwriting /kaggle/working/FPTrans/configs/vision24_vit.yml


In [3]:
%%writefile /kaggle/working/FPTrans/tools.py
# %load /kaggle/working/FPTrans/tools.py
from pathlib import Path

import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
from sacred import Experiment
from scipy.ndimage import distance_transform_edt
import cv2

from config import MapConfig
from data_kits.voc_coco import SemData
from data_kits.datasets import DATA_DIR, DATA_LIST
from utils_.misc import load_image

ex = Experiment("Tools", save_git_info=False, base_dir="./")


@ex.config
def config():
    dataset = "PASCAL"
    proc = "cv2"
    sigma = 5
    save_byte = False       # save weights in uint8 format for saving space (especially for COCO)
    weights_save_dir = DATA_DIR[dataset] / "weights"
    dry_run = False


def boundary2weight(target, cls, kernel, sigma=5):
    mask = torch.zeros(target.shape, dtype=torch.float32).cuda()
    mask[target == cls] = 1
    mask.unsqueeze_(dim=0)      # [1, H, W]
    mask.unsqueeze_(dim=0)      # [1, 1, H, W]

    # Extract mask boundary (inner and outer)
    dilated = torch.clamp(F.conv2d(mask, kernel, padding=1), 0, 1) - mask
    erosion = mask - torch.clamp(F.conv2d(mask, kernel, padding=1) - 8, 0, 1)
    boundary = (dilated + erosion).squeeze(dim=0).squeeze(dim=0)     # [H, W]

    bool_boundary = np.around(boundary.cpu().numpy()).astype(bool)
    edt = distance_transform_edt(np.bitwise_not(bool_boundary))
    weight = np.exp(-edt / sigma ** 2).astype(np.float32)
    return weight


@ex.command(unobserved=True)
def precompute_loss_weights(_config):
    """

    Precompute weights for weighted cross-entropy loss.

    Parameters
    ----------
    _config: ReadOnlyDict
        dataset: str
            Name of the dataset. [PASCAL|COCO | VISION24 ]
        save_bytes: bool
            Save weights in float32 or byte. It should be set as True when generating
            for COCO. Default value is False.
    Returns
    -------

    Usage
    -----
    cuda 0 python tools.py precompute_loss_weights with dataset=PASCAL
    cuda 0 python tools.py precompute_loss_weights with dataset=COCO save_bytes=True
    cuda 0 python tools.py precompute_loss_weights with dataset=VISION24

    """
    opt = MapConfig(_config)
    save_dir = opt.weights_save_dir
    save_dir.mkdir(parents=True, exist_ok=True)
    data_dir = DATA_DIR[opt.dataset]

    # kernel for finding mask boundaries
    kernel = torch.ones(1, 1, 3, 3, dtype=torch.float).cuda()

    data_list = DATA_LIST[opt.dataset]['train']

    label_paths = [x.split()[1] for x in data_list.read_text().splitlines()]

    #for vision24
    all_labels = {}
    if(opt.dataset== "VISION24"):
        all_labels = {str(data_dir/(x.split()[1])):SemData.class_names['vision24'].index(x.split()[2]) for x in data_list.read_text().splitlines()}
    

    gen = tqdm(label_paths)
    for lab_path in gen:
        save_file = save_dir / Path(lab_path).stem
        gen.set_description(f"{save_file}.npz")
        if save_file.with_suffix('.npz').exists():
            continue

        classes = []
        all_class_edts = []

        lab_path = data_dir / lab_path
        label = load_image(lab_path, 'lab', opt.proc)
        
        if(opt.dataset!= "VISION24"):
            unique_labels = np.unique(label).tolist()
            unique_labels = [1 if x == 255 else x for x in unique_labels]
        else:
            unique_labels = all_labels[str(lab_path)]
            unique_labels = np.unique(unique_labels).tolist()
                      
        for cls in unique_labels:
            if cls == 255 :
                continue
            classes.append(cls)
            all_class_edts.append(boundary2weight(label, cls, kernel, opt.sigma))

        classes = np.array(classes)
        edt = np.stack(all_class_edts, axis=0)
        
        if opt.save_byte:
            edt = (edt * 255).astype('uint8')

        if not opt.dry_run:
            np.savez_compressed(save_file, x=edt, c=classes)


@ex.command(unobserved=True)
def print_ckpt(ckpt):
    """

    This tool helps print the weight names and shapes for inspecting a checkpoint.

    Parameters
    ----------
    ckpt: str
        Path to a checkpoint

    """
    state = torch.load(ckpt, map_location='cpu')
    if 'model_state' in state:
        state = state['model_state']
    elif 'state_dict' in state:
        state = state['state_dict']
    elif 'model' in state:
        state = state['model']

    max_name_length = max([len(x) for x in state])
    max_shape_length = max([len(str(x.shape)) for x in state.values()])
    pattern = "  {:<%ds}  {:<%ds}" % (max_name_length, max_shape_length)

    print_str = ""
    for k, v in state.items():
        print_str += pattern.format(k, str(list(v.shape))) + "\n"

    print(print_str)


@ex.command(unobserved=True)
def gen_coco_labels(sets, _config):
    """

    Generate COCO labels with 'pycocotools' API.

    Parameters
    ----------
    sets: str
        Data sets. The accessible values are [train2014, val2014].
    _config: ReadOnlyDict
        dry_run: bool
            Dry run this command without saving to disk.

    Returns
    -------

    Usage
    -----
    python tools.py gen_coco_labels with sets=train2014
    python tools.py gen_coco_labels with sets=val2014

    """
    from pycocotools.coco import COCO

    opt = MapConfig(_config)
    if sets not in ['train2014', 'val2014']:
        raise ValueError(f'Not supported sets: {sets}. [train2014, val2014]')
    save_dir = DATA_DIR['COCO'] / f'{sets}_label'
    annFile = DATA_DIR['COCO'] / f'annotations/instances_{sets}.json'
    save_dir.mkdir(parents=True, exist_ok=True)
    print(f'Labels of {sets} are saved to {save_dir}.')

    coco = COCO(str(annFile))
    # display COCO categories and supercategories
    cats = coco.loadCats(coco.getCatIds())

    nms = [cat['name'] for cat in cats]
    num_cats = len(nms)
    print('All {} categories.'.format(num_cats))
    print(nms)

    # get all images ids
    imgIds = coco.getImgIds()
    gen = tqdm(enumerate(imgIds), total=len(imgIds))
    for idx, im_id in gen:
        # load annotations
        annIds = coco.getAnnIds(imgIds=im_id, iscrowd=False)
        if len(annIds) == 0:
            continue

        image = coco.loadImgs([im_id])[0]
        # image.keys: ['coco_url', 'flickr_url', 'date_captured', 'license', 'width', 'height', 'file_name', 'id']
        h, w = image['height'], image['width']
        gt_name = image['file_name'].split('.')[0] + '.png'
        gt = np.zeros((h, w), dtype=np.uint8)

        # ann.keys: ['area', 'category_id', 'bbox', 'iscrowd', 'id', 'segmentation', 'image_id']
        anns = coco.loadAnns(annIds)
        for ann_idx, ann in enumerate(anns):

            cat = coco.loadCats([ann['category_id']])
            cat = cat[0]['name']
            cat = nms.index(cat) + 1  # cat_id ranges from 1 to 80

            # below is the original script
            segs = ann['segmentation']
            for seg in segs:
                seg = np.array(seg).reshape(-1, 2)  # [n_points, 2]
                cv2.fillPoly(gt, [seg.astype(np.int32)], int(cat))

        save_gt_path = save_dir / gt_name
        gen.set_description(f'{save_gt_path}')
        if not opt.dry_run:
            cv2.imwrite(str(save_gt_path), gt)


if __name__ == "__main__":
    ex.run_commandline()


Overwriting /kaggle/working/FPTrans/tools.py


In [4]:
%%writefile /kaggle/working/FPTrans/data_kits/voc_coco.py
# %load  data_kits/voc_coco.py
import pickle
from pathlib import Path

import cv2
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset
from tqdm import tqdm
import os
import numpy as np
from constants import project_dir, data_dir
from utils_.misc import load_image, load_weights

cache_image = {}
cache_label = {}
cache_weights = {}


def make_dataset(split=0, data_root=None, data_list=None, sub_list=None, coco2pascal=False):
    assert split in [0, 1, 2, 3]
    data_list = Path(data_list).relative_to(project_dir)
    if not data_list.is_file():
        raise RuntimeError(f"Image list file do not exist: {data_list}\n")
    print('begin')
    filepath = data_list.parent / f"{data_list.stem}_{split}{'_coco2pascal' if coco2pascal else ''}.pkl"
    if filepath.exists():
        # Try to load `image_label_list` and `sub_class_file_list` from cache
        # print(filepath)
        with filepath.open("rb") as f:
            image_label_list, sub_class_file_list = pickle.load(f)
            
            # print(image_label_list)
            # print('----------------------------------')

        if len(image_label_list) > 0:
            return image_label_list, sub_class_file_list

    # Shaban uses these lines to remove small objects:
    # if util.change_coordinates(mask, 32.0, 0.0).sum() > 2:
    #    filtered_item.append(item)
    # which means the mask will be downsampled to 1/32 of the original size and the valid area should be larger than 2,
    # therefore the area in original size should be accordingly larger than 2 * 32 * 32
    image_label_list = []
    list_read = open(data_list).readlines()
    print("Processing data...".format(sub_list))
    sub_class_file_list = {}

    # For file name --> if vision24
    dataset_name = str(filepath).split('/')[-2]
    for sub_c in sub_list:
        sub_class_file_list[sub_c] = []

    for l_idx in tqdm(range(len(list_read))):
        line = list_read[l_idx]
        line = line.strip()
        line_split = line.split(' ')
        image_name = data_root / line_split[0]
        label_name = data_root / line_split[1]
        
        label = cv2.imread(str(label_name), cv2.IMREAD_GRAYSCALE)
        label_class = np.unique(label).tolist()
        
        if(dataset_name == "vision24"):
            #read the label from the file
            label_class = line_split[2]
            label_class = [SemData.class_names['vision24'].index(str(label_class))]
            

        
            


        # if 0 in label_class:
        #     label_class.remove(0)
        # if 255 in label_class:
        #     label_class.remove(255)

        # print('-----------------')
        # print(label_class)
        # print('---------------')
        
        #replace each value 255 with 1
        
        
        
        new_label_class = []
        raw_label_list = []
        if(dataset_name == "vision24"):  
            pass
            # sub_list = [255 if x == 1 else x for x  in sub_list]
            
        
        
        for c in label_class:
            
            if c in sub_list:
                # c = 255 if c == 1 else c
                raw_label_list.append(c)
                # check the area of the mask
                tmp_label = np.zeros_like(label)
                
                target_pix = np.where(label == c)
               
                tmp_label[target_pix[0], target_pix[1]] = 1
                if tmp_label.sum() >= 2 * 32 * 32:
                    new_label_class.append(c)


        label_class = new_label_class
        
        # print('label class')
        # print('--------------------')
        # print(label_class)
        # print('--------------------')
            
            
        if(dataset_name == "vision24"):  
            #read the label from the file
            label_class = line_split[2]
            label_class = [SemData.class_names['vision24'].index(str(label_class))]
            sub_list = [1 if x == 255 else x for x  in sub_list]
            raw_label_list = [1 if x ==255 else x for x in raw_label_list]
            label_class = [1 if x == 255 else x for x in label_class]
        
        
        item = (
            image_name.relative_to(data_dir),
            label_name.relative_to(data_dir),
            raw_label_list
        )
        if len(label_class) > 0:
            image_label_list.append(item)
            if(dataset_name != "vision24"):
                #if not vision24 
                for c in label_class:
                    if c in sub_list:
                        sub_class_file_list[c].append(item)
            else:
                for c in label_class:
                    sub_class_file_list[c].append(item)
    
    with filepath.open("wb") as f:
        pickle.dump((image_label_list, sub_class_file_list), f)
    # print('----------')
    # print(image_label_list)
    # print(sub_class_file_list)
    # print('-----------------')
    print(f"Checking image&label list done! There are {len(image_label_list)} images in split {split}.")
    # print('sub_class_file_list')
    # print(sub_class_file_list.keys())
    
    return image_label_list, sub_class_file_list


class SemData(Dataset):
    class_names = {
        "pascal": ["background",
                   "airplane", "bicycle", "bird", "boat", "bottle",
                   "bus", "car", "cat", "chair", "cow",
                   "dining table", "dog", "horse", "motorbike", "person",
                   "potted plant", "sheep", "sofa", "train", "tv"
                   ],
        "coco": ["background",
                 "person", "airplane", "boat", "parking meter", "dog", "elephant", "backpack",
                 "suitcase", "sports ball", "skateboard", "wine glass", "spoon", "sandwich", "hot dog",
                 "chair", "dining table", "mouse", "microwave", "refrigerator", "scissors",
                 "bicycle", "bus", "traffic light", "bench", "horse", "bear", "umbrella",
                 "frisbee", "kite", "surfboard", "cup", "bowl", "orange", "pizza",
                 "couch", "toilet", "remote", "oven", "book", "teddy bear",
                 "car", "train", "fire hydrant", "bird", "sheep", "zebra", "handbag",
                 "skis", "baseball bat", "tennis racket", "fork", "banana", "broccoli", "donut",
                 "potted plant", "tv", "keyboard", "toaster", "clock", "hair drier",
                 "motorcycle", "truck", "stop sign", "cat", "cow", "giraffe", "tie",
                 "snowboard", "baseball glove", "bottle", "knife", "apple", "carrot", "cake",
                 "bed", "laptop", "cell phone", "sink", "vase", "toothbrush",
                 ],
        # "vision24":["defect" , "non-defect"],
        "vision24" : ["Cable_thunderbolt" , "Cable_torn_apart" , "Cylinder_Chip" , "Cylinder_Porosity" ,"Cylinder_RCS",
                          "PCB_mouse_bite" , "PCB_open_circuit" , "PCB_short" , "PCB_spur" , "PCB_spurious_copper" , "Screw_front",
                          "Wood_impurities"]
 
        
    }

    def __init__(self, opt, split, shot, query,
                 data_root=None, data_list=None, transform=None, mode='train',
                 cache=True):
        assert mode in ['train', 'val', 'test', 'eval_online']

        if mode != "train" and opt.dataset in ["PASCAL", "COCO" , "VISION24"]:
            mode = "val"
        self.opt = opt
        self.mode = mode
        self.split = split
        self.shot = shot
        self.query = query
        self.data_root = Path(data_root)
        self.tasks = []
        self.transform = transform
        self.cache = cache

        if opt.dataset == "PASCAL":
            n_class = 20
            interval = 5
            self.class_list = list(range(1, 21))
            self.sub_val_list = list(range(interval * split + 1, interval * (split + 1) + 1))
            self.sub_list = list(set(range(1, n_class + 1)) - set(self.sub_val_list))
            
            if opt.coco2pascal:
                n_class = 20
                self.class_list = list(range(1, 21))
                if split == 0:
                    self.sub_val_list = [1, 4, 9, 11, 12, 15]
                elif split == 1:
                    self.sub_val_list = [2, 6, 13, 18]
                elif split == 2:
                    self.sub_val_list = [3, 7, 16, 17, 19, 20]
                elif split == 3:
                    self.sub_val_list = [5, 8, 10, 14]
                self.sub_list = list(set(range(1, 21)) - set(self.sub_val_list))

        elif opt.dataset == "COCO":
            n_class = 80
            if opt.use_split_coco:
                print('INFO: using SPLIT COCO')
                self.class_list = list(range(1, 81))
                self.sub_val_list = list(range(split + 1, 81, 4))
                self.sub_list = list(set(self.class_list) - set(self.sub_val_list))
            else:
                interval = 20
                print('INFO: using COCO')
                self.class_list = list(range(1, 81))
                self.sub_val_list = list(range(interval * split + 1, interval * (split + 1) + 1))
                self.sub_list = list(set(range(1, n_class + 1)) - set(self.sub_val_list))
        elif opt.dataset == "VISION24":
            n_class =12;
            interval = 2
            # self.class_list =[ i for i in range(len(self.class_names['vision24']))] 
            # self.sub_val_list =  [ i for i in range(len(self.class_names['vision24']))]
            # self.sub_list = [ i for i in range(len(self.class_names['vision24']))]

            self.class_list = [i for i in range(n_class)]
            self.sub_val_list = [i for i in range(n_class)]
            self.sub_list = [i for i in range(n_class)]
            
            # print('1--------------------------------')
            # print(self.sub_val_list)
            # print('\n\n')
            # print('2--------------------------------')
            # print(self.sub_list)
        else:
            raise ValueError(f'Not supported dataset: {opt.dataset}. [PASCAL|COCO | VISION24]')

        self.n_class = n_class
        self.train_num_classes = len(self.sub_list)
        if self.mode == 'train':
            self.data_list, self.sub_cls_files = make_dataset(
                split, self.data_root, data_list, self.sub_list)
            
        else:
            self.data_list, self.sub_cls_files = make_dataset(
                split, self.data_root, data_list, self.sub_val_list, opt.coco2pascal)

        # print('----------------------------------')
        # print(self.data_list)
        # print(type(self.data_list))
        # print('-------------------------')
        for c in self.sub_cls_files:
            print(f"Class {c}: {len(self.sub_cls_files[c])} images")

        self.length_data_list = len(self.data_list)
        print(f"Data list length: {self.length_data_list}")
        assert self.length_data_list > 0, 'Length of data list is 0. Please make sure the data root ' \
                                          f'({self.data_root}) and data list ({data_list}) are correct.'
        self.length_sub_class_list = {k: len(v) for k, v in self.sub_cls_files.items()}

        self.reset_sampler()

    def __len__(self):
        if self.mode == 'train':
            return self.opt.train_n or self.length_data_list
        else:
            return self.opt.test_n or self.length_data_list

    def reset_sampler(self):
        seed = self.opt.seed
        test_seed = self.opt.test_seed
        # Use fixed test sampler(opt.test_seed) for reproducibility
        self.sampler = np.random.RandomState(seed) \
            if self.mode == "train" else np.random.RandomState(test_seed)

    def sample_tasks(self):
        self.tasks = []

        total_length = len(self)
        rounds = (total_length + self.length_data_list - 1) // self.length_data_list
        counter = 0
        
        for r in range(rounds):
            # Sampling random episodes. Use self.reset_sampler() for fixed sampling orders
            rng = self.sampler.permutation(np.arange(self.length_data_list)) 
            for idx in rng:
                item = self.data_list[idx]
                
                assert len(item[2]) > 0

                cls = self.sampler.choice(item[2])
                
                num_files = len(self.sub_cls_files[cls])
                
               
                s_indices = []
                for i in range(self.shot):
                    while True:
                        # print(f"Class {cls}: {self.sub_cls_files.get(cls, 'Not Found')}")
                        

                        s_idx = self.sampler.choice(num_files, size=1)[0]
                        
                        if self.sub_cls_files[cls][s_idx] == item or s_idx in s_indices:
                            continue
                        s_indices.append(s_idx)
                        break
                self.tasks.append((idx, cls, s_indices))
            
            
                counter += 1
                if counter >= total_length:
                    break
       
            
            

    def seg_encode(self, lab, cls, ignore_lab):
        if self.opt.proc == 'pil':
            lab = np.array(lab, np.uint8)
        target_pix = np.where(lab == cls)
        ignore_pix = np.where(lab == 255)
        lab[:, :] = 0
        if target_pix[0].shape[0] > 0:
            lab[target_pix[0], target_pix[1]] = 1
        if ignore_pix[0].shape[0] > 0:
            lab[ignore_pix[0], ignore_pix[1]] = ignore_lab
        if self.opt.proc == 'pil':
            lab = Image.fromarray(lab)
        return lab
    # def seg_encode(self, lab, cls, ignore_lab):
    #     if self.opt.proc == 'pil':
    #         lab = np.array(lab, np.uint8)
    
    #     target_pix = np.where(lab == 255)  # Find all pixels with 255
    #     background_pix = np.where(lab == 0)
    
    #     lab[:, :] = 0  # Set all pixels to background (0)
    #     if target_pix[0].shape[0] > 0:
    #         lab[target_pix[0], target_pix[1]] = 1  # Convert 255 → 1 for class index
    #     if ignore_pix[0].shape[0] > 0:
    #         lab[ignore_pix[0], ignore_pix[1]] = ignore_lab
    
    #     if self.opt.proc == 'pil':
    #         lab = Image.fromarray(lab)
    
    #     return lab


    def get_image(self, name, cache=True):
        if self.cache and cache:
            if name not in cache_image:
                cache_image[name] = load_image(data_dir / name, 'img', self.opt.proc)
            return cache_image[name].copy()
        else:
            return load_image(data_dir / name, 'img', self.opt.proc)

    def get_label(self, name, cls, ignore_lab=0, cache=True):
        if self.cache and cache:
            if name not in cache_label:
                cache_label[name] = load_image(data_dir / name, 'lab', self.opt.proc)
            lab = cache_label[name].copy()
        else:
            lab = load_image(data_dir / name, 'lab', self.opt.proc)
        lab = self.seg_encode(lab, cls, ignore_lab)
        # Convert 255 to 1
        
        lab = np.where(lab == 255, 1, 0).astype(np.uint8)
        return lab

    def get_weights(self, name, cls, cache=True):
        if self.cache and cache:
            if name not in cache_weights:
                cache_weights[name] = load_weights(data_dir / name)
            weights_dict = cache_weights[name]
        else:
            weights_dict = load_weights(data_dir / name)
        if cls in weights_dict['c']:
            class_index = weights_dict['c'].index(cls)
            weights = weights_dict['x'][class_index].copy().astype(np.float32)
        else:
            weights = np.zeros(weights_dict['x'][0].shape, np.float32)
        return weights
    
    # def get_weights(self, name, cls, cache=True):
    #     weight_path = data_dir / name
    
    #     # If file doesn't exist, return default zero weights
    #     if not os.path.exists(weight_path):
    #         print(f"Warning: Weight file {weight_path} not found. Using zero weights.")
    #         return np.zeros((1, 768), np.float32)  # Adjust shape as needed
    
    #     if self.cache and cache:
    #         if name not in cache_weights:
    #             cache_weights[name] = load_weights(weight_path)
    #         weights_dict = cache_weights[name]
    #     else:
    #         weights_dict = load_weights(weight_path)
    
    #     if cls in weights_dict['c']:
    #         class_index = weights_dict['c'].index(cls)
    #         weights = weights_dict['x'][class_index].copy().astype(np.float32)
    #     else:
    #         weights = np.zeros(weights_dict['x'][0].shape, np.float32)
    
    #     return weights

    def __getitem__(self, index):
        qry_idx, cls, support_indices = self.tasks[index]
        image_path, label_path, _ = self.data_list[qry_idx]
        sup_image_paths = []
        sup_label_paths = []
       
        for sup_idx in support_indices:
            x_path, y_path, _ = self.sub_cls_files[cls][sup_idx]
            sup_image_paths.append(x_path)
            sup_label_paths.append(y_path)

        
        qry_names = [image_path.stem]
        sup_names = [x.stem for x in sup_image_paths]

        image = self.get_image(image_path)
        # keep query ignore_label as 255, which has significant to iou.
        label = self.get_label(label_path, cls, ignore_lab=255)
        kwargs = {}
        sup_kwargs = [{} for _ in range(self.opt.shot)]
        # Load weights
        if self.mode == 'train':
            if self.opt.precompute_weight and self.opt.loss == 'cedt':
                # Load precomputed weights, used for computing loss
                weight_path = image_path.parents[1] / f"weights/{image_path.stem}.npz"
                kwargs['weights'] = self.get_weights(weight_path, cls)


        
        sup_images = [self.get_image(x) for x in sup_image_paths]
        sup_labels = [self.get_label(x, cls, ignore_lab=255) for x in sup_label_paths]

        # raw_label = label.copy()
        if self.transform is not None:
            image, label, kwargs = self.transform[1](image, label, **kwargs)
            for k in range(self.shot):
                sup_images[k], sup_labels[k], sup_kwargs[k] = self.transform[0](sup_images[k], sup_labels[k],
                                                                                **sup_kwargs[k])

        sup_images = torch.stack(sup_images, dim=0)
        sup_labels = torch.stack(sup_labels, dim=0)
        label = label.unsqueeze(dim=0)

        ret_dict = {
            'sup_rgb': sup_images,  # [S, 3, H, W]
            'sup_msk': sup_labels,  # [S, H, W]
            'qry_rgb': image,  # [3, H, W]
            'qry_msk': label,  # [1, H, W]
            'cls': cls,  # [], values in [1, 20] for PASCAL
            'weights': kwargs.get('weights', None),
            'sup_names': sup_names,
            'qry_names': qry_names,
        }
        ret_dict = {k: v for k, v in ret_dict.items() if v is not None}
        return ret_dict

    def get_class_name(self, cls, dataset):
        return self.class_names[dataset.lower()][cls]


Overwriting /kaggle/working/FPTrans/data_kits/voc_coco.py


In [5]:
%%writefile /kaggle/working/FPTrans/data_kits/datasets.py
# %load /kaggle/input/fptrans_vision24/pytorch/default/1/FPTrans/data_kits/datasets.py
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader

from constants import data_dir, lists_dir
from data_kits import transformation as tf
from data_kits import voc_coco as pfe
from utils_.misc import load_image
from data_kits.binary_dataset import BinarySegmentationDataset


DATA_DIR = {
    "PASCAL": data_dir / "VOCdevkit/VOC2012",
    "COCO": data_dir / "COCO",
    "VISION24": data_dir / "VISION24",
     
}
DATA_LIST = {
    "PASCAL": {
        "train": lists_dir / "pascal/voc_sbd_merge_noduplicate.txt",
        "test": lists_dir / "pascal/val.txt",
        "eval_online": lists_dir / "pascal/val.txt"
    },
    "COCO": {
        "train": lists_dir / "coco/train_data_list.txt",
        "test": lists_dir / "coco/val_data_list.txt",
        "eval_online": lists_dir / "coco/val_data_list.txt"
    },
    "VISION24": {
        "train": lists_dir / "vision24/train.txt",
        "test": lists_dir / "vision24/test.txt",
        "eval_online": lists_dir / "vision24/test.txt"
    },
    
}


MEAN = [0.485, 0.456, 0.406]    # list, normalization mean in data preprocessing
STD = [0.229, 0.224, 0.225]     # list, normalization std in data preprocessing


def get_train_transforms(opt, height, width):
    supp_transform = tf.Compose([tf.RandomResize(opt.scale_min, opt.scale_max),
                                 tf.RandomRotate(opt.rotate, pad_type=opt.pad_type),
                                 tf.RandomGaussianBlur(),
                                 tf.RandomHorizontallyFlip(),
                                 tf.RandomCrop(height, width, check=True, center=True, pad_type=opt.pad_type),
                                 tf.ToTensor(mask_dtype='float'),   # support mask using float
                                 tf.Normalize(MEAN, STD)], processer=opt.proc)

    query_transform = tf.Compose([tf.RandomResize(opt.scale_min, opt.scale_max),
                                  tf.RandomRotate(opt.rotate, pad_type=opt.pad_type),
                                  tf.RandomGaussianBlur(),
                                  tf.RandomHorizontallyFlip(),
                                  tf.RandomCrop(height, width, check=True, center=True, pad_type=opt.pad_type),
                                  tf.ToTensor(mask_dtype='long'),   # query mask using long
                                  tf.Normalize(MEAN, STD)], processer=opt.proc)

    return supp_transform, query_transform


def get_val_transforms(opt, height, width):
    supp_transform = tf.Compose([tf.Resize(height, width),
                                 tf.ToTensor(mask_dtype='float'),   # support mask using float
                                 tf.Normalize(MEAN, STD)], processer=opt.proc)

    query_transform = tf.Compose([tf.Resize(height, width, do_mask=False),  # keep mask the original size
                                  tf.ToTensor(mask_dtype='long'),   # query mask using long
                                  tf.Normalize(MEAN, STD)], processer=opt.proc)

    return supp_transform, query_transform


def load(opt, logger, mode):
    split, shot, query = opt.split, opt.shot, 1
    height, width = opt.height, opt.width

    if mode == "train":
        data_transform = get_train_transforms(opt, height, width)
    elif mode in ["test", "eval_online", "predict"]:
        data_transform = get_val_transforms(opt, height, width)
    else:
        raise ValueError(f'Not supported mode: {mode}. [train|eval_online|test|predict]')

    if opt.dataset == "PASCAL":
        num_classes = 20
        cache = True
    elif opt.dataset == "COCO":
        num_classes = 80
        cache = False
    elif opt.dataset == "VISION24":
        num_classes = 12
        cache = False
    else:
        raise ValueError(f'Not supported dataset: {opt.dataset}. [PASCAL|COCO | VISION24]')

    dataset = pfe.SemData(opt, split, shot, query,
                          data_root=DATA_DIR[opt.dataset],
                          data_list=DATA_LIST[opt.dataset][mode],
                          transform=data_transform,
                          mode=mode,
                          cache=cache)

    dataloader = DataLoader(dataset,
                            batch_size=opt.bs if mode == 'train' else opt.test_bs,
                            shuffle=True if mode == 'train' else False,
                            num_workers=opt.num_workers,
                            pin_memory=True,
                            drop_last=True if mode == 'train' else False )

    logger.info(' ' * 5 + f"==> Data loader {opt.dataset} for {mode}")
    return dataset, dataloader, num_classes


def get_val_labels(opt, mode):
    if opt.dataset == "PASCAL":
        if opt.coco2pascal:
            if opt.split == 0:
                sub_val_list = [1, 4, 9, 11, 12, 15]
            elif opt.split == 1:
                sub_val_list = [2, 6, 13, 18]
            elif opt.split == 2:
                sub_val_list = [3, 7, 16, 17, 19, 20]
            elif opt.split == 3:
                sub_val_list = [5, 8, 10, 14]
            else:
                raise ValueError(f'PASCAL only have 4 splits [0|1|2|3], got {opt.split}')
        else:
            sub_val_list = list(range(opt.split * 5 + 1, opt.split * 5 + 6))
        return sub_val_list
    elif opt.dataset == "COCO":
        if opt.use_split_coco:
            return list(range(opt.split + 1, 81, 4))
        return list(range(opt.split * 20 + 1, opt.split * 20 + 21))
    elif opt.dataset == "VISION24":
        # return list(range(opt.split + 1, 6))
        return [0 , 1 , 2 , 3  ,4 , 5]
    else:
        raise ValueError(f'Only support datasets [PASCAL|COCO|VISION24], got {opt.dataset}')


def load_p(opt, device):
    supp_t, query_t = get_val_transforms(opt, opt.height, opt.width)
    p = opt.p

    if p.sup and p.qry:
        supp_rgb_path = DATA_DIR[opt.dataset] / "JPEGImages" / f"{p.sup}.jpg"
        supp_lab_path = DATA_DIR[opt.dataset] / "SegmentationClassAug" / f"{p.sup}.png"
        query_rgb_path = DATA_DIR[opt.dataset] / "JPEGImages" / f"{p.qry}.jpg"
        query_lab_path = DATA_DIR[opt.dataset] / "SegmentationClassAug" / f"{p.qry}.png"

        supp_rgb = load_image(supp_rgb_path, 'img', opt.proc)
        _supp_lab = load_image(supp_lab_path, 'lab', opt.proc)
        supp_lab = np.zeros_like(_supp_lab, dtype=_supp_lab.dtype)
        supp_lab[_supp_lab == 255] = 255
        supp_lab[_supp_lab == p.cls] = 1
        query_ori = query_rgb = load_image(query_rgb_path, 'img', opt.proc)
        query_lab = np.zeros(query_rgb.shape[:-1], dtype=_supp_lab.dtype)
        _query_lab = load_image(query_lab_path, 'lab', opt.proc)
        query_lab[_query_lab == 255] = 255
        query_lab[_query_lab == p.cls] = 1

        supp_img, supp_lab, _ = supp_t(supp_rgb, supp_lab)
        query_img, query_lab, _ = query_t(query_rgb, query_lab)

        supp_img = supp_img[None, None].to(device)      # [B, S, 3, H, W]
        supp_lab = supp_lab[None, None].to(device)      # [B, S, H, W]
        query_img = query_img[None].to(device)          # [B, 3, H, W]
        query_lab = query_lab[None].to(device)      # [B, H, W]
    elif p.sup_rgb and p.sup_msk and p.qry_rgb:
        _supp_rgbs = [load_image(x, 'img', opt.proc) for x in p.sup_rgb]
        _supp_labs = [load_image(x, 'lab', opt.proc, mode=cv2.IMREAD_UNCHANGED) for x in p.sup_msk]
        supp_rgbs = []
        supp_labs = []
        for i, _supp_lab in enumerate(_supp_labs):
            if len(_supp_lab.shape) != 2:
                _supp_lab = _supp_lab[:, :, -1]
            supp_lab = np.zeros_like(_supp_lab, dtype=_supp_lab.dtype)
            if p.cls == 255:
                supp_lab[_supp_lab == p.cls] = 1
            else:
                supp_lab[_supp_lab == 255] = 255
                supp_lab[_supp_lab == p.cls] = 1
            supp_img, supp_lab, _ = supp_t(_supp_rgbs[i], supp_lab)
            supp_rgbs.append(supp_img)
            supp_labs.append(supp_lab)
        supp_img = torch.stack(supp_rgbs, dim=0)
        supp_lab = torch.stack(supp_labs, dim=0)

        query_ori = [load_image(x, 'img', opt.proc) for x in p.qry_rgb]
        _query_rgbs = query_ori
        _query_labs = [np.zeros(x.shape[:-1], dtype=_supp_labs[0].dtype) for x in _query_rgbs]
        query_rgbs = []
        for i, _query_lab in enumerate(_query_labs):
            query_img, query_lab, _ = query_t(_query_rgbs[i], _query_lab)
            query_rgbs.append(query_img)
        query_img = torch.stack(query_rgbs, dim=0)

        supp_img = supp_img[None].to(device)            # [B, S, 3, H, W]
        supp_lab = supp_lab[None].to(device)            # [B, S, H, W]
        query_img = query_img.to(device)                # [Q, 3, H, W]
        query_lab = None
    else:
        raise ValueError(f'In the prediction mode, either the [p.sup, p.qry] or the \n'
                         f'[p.sup_rgb, p.sup_msk, p.qry_rgb] should be given. Got \n'
                         f'    p.sup={p.sup}, p.qry={p.qry}\n'
                         f'    p.sup_rgb={p.sup_rgb}, p.sup_msk={p.sup_msk}, p.qry_rgb={p.qry_rgb}')


    return supp_img, supp_lab, query_img, query_lab, query_ori


Overwriting /kaggle/working/FPTrans/data_kits/datasets.py


In [6]:
%cd /kaggle/working/FPTrans

/kaggle/working/FPTrans


In [7]:
%%writefile /kaggle/working/FPTrans/core/base_trainer.py
# %load /kaggle/working/FPTrans/core/base_trainer.py
import os
import random
import shutil
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from tqdm import tqdm

from constants import on_cloud
from core import losses as loss_utils
from core import solver
from core.metrics import FewShotMetric, Accumulator
from data_kits import datasets
from utils_.loggers import C as CC
from utils_.timer import Timer


def round_(array):
    if isinstance(array, float) or array.ndim == 0:
        return f"{array:5.2f}"
    if array.ndim == 1:
        return "[" + ", ".join([f"{x:5.2f}" for x in array]) + "]"


def save_img(file_path, img):
    pil_img = Image.fromarray(img.astype(np.uint8))
    # pil_img.putpalette(palette)
    pil_img.save(file_path)


def pad(str0, padded, length, align='left'):
    remains = length - len(str0)
    if align == 'left':
        left = 1
    elif align == 'center':
        left = remains // 2
    else:
        raise ValueError
    right = remains - left
    return padded * left + str0 + padded * right


class BaseEvaluator(object):
    """
    Evaluator base class. Evaluator is used in the validation stage and testing stage.
    All the evaluators should inherit from this class and implement the `test_step()`
    function.

    Parameters
    ----------
    opt: misc.MapConfig
        Experiment configuration.
    model: nn.Module
        PyTorch model instance.
    mode: str
        Evaluation mode. [EVAL_ONLINE, EVAL]

    """
    def __init__(self, opt, logger, device, model, model_T, mode, need_grad=False):
        self.opt = opt
        self.logger = logger
        self.device = device
        self.mode = mode
        if mode not in ["EVAL_ONLINE", "EVAL"]:
            raise ValueError(f"Not supported evaluation mode {mode}. [EVAL_ONLINE, EVAL]")
        self.num_devices = torch.cuda.device_count()

        self.model = model
        if not isinstance(self.model, nn.DataParallel):
            self.model_DP = self.init_device(self.model)
        else:
            self.model_DP = self.model
        # if model_T is not None:
        #     self.model_T = model_T
        #     if not isinstance(self.model_T, nn.DataParallel):
        #         self.model_T_DP = self.init_device(self.model_T)
        #     else:
        #         self.model_T_DP = self.model_T
        # else:
        #     self.model_T = None
        #     self.model_T_DP = None
        
        self.loss_obj = loss_utils.get(opt, logger, loss=opt.loss.replace('dt', ''))
        self.need_grad = need_grad

    def test_step(self, batch, step):
        raise NotImplementedError

    def init_device(self, net):
        net = net.to(self.device)
        net_DP = nn.DataParallel(net, device_ids=range(self.num_devices))
        return net_DP

    def maybe_print_metrics(self, accu, gen, epoch=None):
        if self.opt.tqdm:
            if epoch is not None:
                print_str = f"[{self.mode}] [{epoch}/{self.opt.epochs}] loss: {accu.mean('loss'):.5f}"
            else:
                print_str = f"[{self.mode}] loss: {accu.mean('loss'):.5f}"
            gen.set_description(print_str)

    @torch.no_grad()
    def start_eval_loop(self, data_loader, num_classes, epoch=None):
        """ For few-shot learning """
        # Set model to evaluation mode (for specific layers, such as batchnorm, dropout, dropblock)
        self.model.eval()
        # Fix sampling order of the test set.
        data_loader.dataset.reset_sampler()
        timer = Timer()
        data_timer = Timer()
        val_labels = datasets.get_val_labels(self.opt, self.mode)
        # create saving directory
        save_dir = None
        if self.opt.save_dir is not None:
            save_dir = Path(self.opt.save_dir)
            save_dir.mkdir(parents=True, exist_ok=True)

        fs_metric = FewShotMetric(num_classes)    
        accu = Accumulator(loss=0)
        data_loader.dataset.sample_tasks()

        gen = data_loader
        if self.opt.tqdm:
            gen = tqdm(gen, leave=True)
        data_timer.tic()
        for i, batch in enumerate(gen, start=1):
            data_timer.toc()
            with timer.start():
                qry_pred, losses = self.test_step(batch, i)
                accu.update(**losses)
            self.maybe_print_metrics(accu, gen, epoch)
            fs_metric.update(qry_pred, batch['qry_msk'], batch['cls'],verbose =0 )

            # save prediction
            if save_dir:
                qry_mask = batch['qry_msk'].cpu().numpy()[:, 0]
                qry_pred = qry_pred[:, 0]
                classes = batch['cls'].cpu().numpy().tolist()
                for j, qry_name in enumerate(batch['qry_names'][0]):
                    if batch['qry_ori_size'] is not None:
                        ori_H, ori_W = batch['qry_ori_size'][j]
                        p = qry_pred[j, :ori_H, :ori_W] * 255
                        r = (qry_mask[j, :ori_H, :ori_W] == 1) * 255
                        save_path = save_dir / (f'{(i - 1) * self.opt.test_bs + j + 1:04d}_' + qry_name + '_pred.png')
                        save_img(save_path, p)
                        save_path = save_dir / (f'{classes[j]:02d}_' + qry_name + '_mask.png')
                        save_img(save_path, r)
            data_timer.tic()

        miou_class, miou_avg , catch_rate , yeild_rate = fs_metric.get_scores(val_labels)
        biou_class, biou_avg , _ , _ = fs_metric.get_scores(val_labels, binary=True)
        str1 = f'mIoU mean: {round_(miou_class * 100)} ==> {round_(miou_avg * 100)}'
        str2 = f'bIoU mean: {round_(biou_class * 100)} ==> {round_(biou_avg * 100)}'

        if self.mode == "EVAL_ONLINE":
            self.logger.info(str1)
            self.logger.info(str2)
        elif self.mode == "EVAL":
            str3 = f'speed: {round_(timer.cps)} FPS'
            max_length = max(len(str1), len(str2), len(str3)) + 2
            self.logger.info('╒' + pad(' Final Results ', '═', max_length, align='center') + '╕')
            self.logger.info('│' + pad(str1, ' ', max_length) + '│')
            self.logger.info('│' + pad(str2, ' ', max_length) + '│')
            self.logger.info('│' + pad(str3, ' ', max_length) + '│')
            self.logger.info('╘' + pad('', '═', max_length) + '╛')

        return accu.mean('loss'), miou_avg, biou_avg, timer.elapsed, data_timer.elapsed , catch_rate, yeild_rate


class BaseTrainer(object):
    def __init__(self, opt, logger, device, model, data_loader, data_loader_val, _run):
        self.opt = opt
        self.logger = logger
        self.device = device
        self.run = _run
        self.data_loader = data_loader
        self.data_loader_val = data_loader_val
        self.num_devices = torch.cuda.device_count()

        # Define model-related objects
        self.model = model
        self.model_DP = self.init_device(self.model)
        self.loss_obj = loss_utils.get(opt, logger)

        self.build_optimizer(verbose=1)
        self.step_lr_counter = 0

        # Define model_dir for saving checkpoints
        self.do_ckpt = True
        self.log_dir = _run.run_dir
        if on_cloud:
            self.cloud_save_dir = Path(f"afs/output/models_fss/{self.log_dir.name}")
            if not self.cloud_save_dir.exists():
                self.cloud_save_dir.mkdir(parents=True)

        # Define metrics and output templates
        self.best_iou = -1.
        self.template = "[{:d}/{:d}]" + \
                        " | Tr {:6.4f} | Val {:6.4f} | mIoU {:5.2f} | bIoU {:5.2f}" + \
                        " | DATA {:s} | OPT {:s} | ETA {:s} | Catch Rate {:5.2f} | Yeild Rate {:5.2f}"

        self.loss_names = 'loss prompt pair'.split()
        self.steps_per_epoch = len(self.data_loader)

    def build_optimizer(self, verbose=0):
        opt = self.opt
        model = self.model

        param_list = model.get_params_list()
        max_steps = opt.epochs * len(self.data_loader)
        self.optimizer, self.scheduler = solver.get(opt, param_list, max_steps)

        if verbose:
            plists = model.get_params_list()
            for plist in plists:
                self.logger.info(f"Number of trainable parameters: {len([_ for _ in plist['params']])}")

    def train_step(self, batch, step, epoch):
        raise NotImplementedError

    def init_device(self, net):
        net = net.to(self.device)
        net_DP = nn.DataParallel(net, device_ids=range(self.num_devices))
        return net_DP

    @staticmethod
    def second2str(seconds):
        seconds = int(seconds)
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds_ = seconds % 60
        return f"{hours:02d}:{minutes:02d}:{seconds_:02d}"

    def time_left(self, iters, epoch, speed, extra_per_epoch=0):
        iters_left = (self.opt.epochs - epoch) * self.steps_per_epoch + (self.steps_per_epoch - iters)
        time_left = int(iters_left * speed) + (self.opt.epochs - epoch) * extra_per_epoch
        return time_left

    def maybe_print_losses(self, iters, epoch, accu, timer, gen, data_timer):
        if self.opt.tqdm:
            fmt_str = "[TRAIN] [{:d}/{:d}] "
            for name in self.loss_names:
                fmt_str += name + ": {:.4f} "
            print_str = fmt_str.format(epoch, self.opt.epochs, *accu.mean(self.loss_names))
            gen.set_description(print_str)
        elif iters % self.opt.print_interval == 0 or iters == self.steps_per_epoch:
            fmt_str = "[{:d}/{:d}] [{:d}/{:d}] "
            for name in self.loss_names:
                fmt_str += name + ": {:.4f} "
            fmt_str += "[SPD: {:.2f}]"
            print_str = fmt_str.format(
                epoch, self.opt.epochs, 
                iters, self.steps_per_epoch, 
                *accu.mean(self.loss_names), 
                timer.spc + data_timer.spc,
            )
            self.logger.info(print_str)

    def print_lr(self):
        for i, group in enumerate(self.optimizer.param_groups):
            self.logger.info(f"Learning rate for {self.opt.optim.upper()} param_group[{i}] is {group['lr']:.6g}")

    def start_training_loop(self, start_epoch, evaluator, num_classes):
        timer = Timer()
        data_timer = Timer()
        accu = Accumulator(**{x: 0. for x in self.loss_names})

        for epoch in range(start_epoch, self.opt.epochs + 1):
            self.print_lr()
            
            # Load previous epoch weights if available
            # if(self.opt.dataset == "VISION24"):
            #     checkpoint_path = os.path.join("vision24", f"epoch_{epoch-1}.pth")
            #     if os.path.exists(checkpoint_path):
            #         print(f"Loading weights from {checkpoint_path}...")
            #         checkpoint = torch.load(checkpoint_path)
            #         self.model.load_state_dict(checkpoint['model_state_dict'])
            #         self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            #         accu.update(**checkpoint['accu'])  # Restore accumulated loss
            

            # 1. Training
            self.model.train()
            self.data_loader.dataset.sample_tasks()
            gen = self.data_loader
            if self.opt.tqdm:
                gen = tqdm(gen, leave=True)

            for i, batch in enumerate(gen, start=1):
                if i > 1:
                    data_timer.toc()
                with timer.start():
                    losses = self.train_step(batch, i, epoch)
                    accu.update(**losses)
                self.maybe_print_losses(i, epoch, accu, timer, gen, data_timer)
                self.step_lr()
                data_timer.tic()

            # 2. Evaluation
            if self.opt.ckpt_interval > 0 and epoch % self.opt.ckpt_interval == 0 or epoch == self.opt.epochs:
                mloss, miou, biou, val_elapsed, val_data_elapsed , catch_rate ,yeild_rate = evaluator.start_eval_loop(self.data_loader_val, num_classes, epoch)
                best = self.snapshot(epoch, miou, biou)
                
                data_time = data_timer.elapsed + val_data_elapsed
                epoch_time = timer.elapsed + val_elapsed
                self.log_result(epoch, accu, mloss, miou, biou, best, epoch_time, data_time , catch_rate , yeild_rate)
                
            # Save model checkpoint after every epoch
            # if(self.opt.dataset == "VISION24"):
            #     checkpoint_path = os.path.join('vision24', f"epoch_{epoch}.pth")
            #     torch.save({
            #         'epoch': epoch,
            #         'model_state_dict': self.model.state_dict(),
            #         'optimizer_state_dict': self.optimizer.state_dict(),
            #         'accu': accu.get(),  # Save accumulated loss
            #     }, checkpoint_path)

            #     print(f"Checkpoint saved: {checkpoint_path}")

            # 3. Prepare for next epoch
            timer.reset()
            data_timer.reset()
            accu.reset()

    def step_lr(self):
        """
        Update learning rate by the specified learning rate policy.
        For 'cosine' and 'poly' policies, the learning rate is updated by steps.
        For other policies, the learning rate is updated by epochs.
        """
        if self.scheduler is None:
            return

        self.step_lr_counter += 1

        if self.opt.lrp in ["cosine", "poly", "cosinev2"]:   # forward per step
            self.scheduler.step()
        elif self.step_lr_counter == self.steps_per_epoch:      # forward per epoch
            self.scheduler.step()
            self.step_lr_counter = 0

    def get_weights(self):
        state_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if 'original_encoder' not in k}
        return state_dict

    def snapshot(self, epoch, miou, biou, verbose=0):
        best = False
        if on_cloud and self.log_dir.stem == "None":
            if miou > self.best_iou:
                best = True
                self.best_iou = miou
            return best

        if miou > self.best_iou:
            best = True
            self.best_iou = miou

        save_path = self.log_dir / "ckpt.pth"
        state = {
            'epoch': epoch,
            'miou': miou,
            'biou': biou,
            'best_miou': self.best_iou,
            'model_state': self.get_weights(),
        }

        torch.save(state, save_path)
        if verbose:
            self.logger.info(CC.c(f" \\_/ Save checkpoint to {save_path}", CC.OKGREEN))

        if best:
            shutil.copyfile(save_path, self.log_dir / "bestckpt.pth")
        
        # Make a copy when oncloud
        if on_cloud:
            state.update({
                'random_state': random.getstate(),
                'np_random_state': np.random.get_state(),
                'torch_random_state': torch.get_rng_state().numpy(),
                'train_sampler_state': self.data_loader.dataset.sampler.get_state(),
                'val_sampler_state': self.data_loader_val.dataset.sampler.get_state(),
                'optimizer_state': self.optimizer.state_dict(),
                'scheduler_state': self.scheduler.state_dict(),
            })

            try:
                on_cloud_save_path = self.cloud_save_dir / "ckpt.pth"
                if on_cloud_save_path.exists():
                    os.remove(on_cloud_save_path)
                torch.save(state, on_cloud_save_path)
            except Exception as e:
                print(e)
            
            if best:
                try:
                    on_cloud_save_path_best = self.cloud_save_dir / "bestckpt.pth"
                    if on_cloud_save_path_best.exists():
                        os.remove(on_cloud_save_path_best)
                    shutil.copyfile(save_path, on_cloud_save_path_best)
                except Exception as e:
                    print(e)
        return best

    def log_result(self, epoch, accu, val_loss, val_mIoU, val_bIoU, best, epoch_time, data_time , catch_rate , yeild_rate, **kwargs):
        # Log epoch summary to the terminal
        losses = accu.mean('loss')
        log_str = self.template.format(
            epoch, 
            self.opt.epochs, 
            losses, 
            val_loss, 
            val_mIoU * 100, 
            val_bIoU * 100, 
            self.second2str(data_time),
            self.second2str(epoch_time),
            self.second2str((self.opt.epochs - epoch) * epoch_time),
            catch_rate * 100,
            yeild_rate * 100
            
        ) + " | (best)" * best
        self.logger.info(log_str)
        self.logger.info(CC.c(f"[{epoch}/{self.opt.epochs}] Best mIoU until now: {self.best_iou * 100:.2f}\n", CC.OKGREEN))

        # Log results to the sacred database
        for k, v in accu.mean(self.loss_names, dic=True).items():
            self.run.log_scalar(k, float(v), epoch)
        self.run.log_scalar('val_loss', float(val_loss), epoch)
        self.run.log_scalar('val_mIoU', float(val_mIoU), epoch)
        self.run.log_scalar('val_bIoU', float(val_bIoU), epoch)
        for k, v in kwargs.items():
            self.run.log_scalar(k, float(v), epoch)
    

Overwriting /kaggle/working/FPTrans/core/base_trainer.py


In [8]:
%%writefile /kaggle/working/FPTrans/core/metrics.py

# %load /kaggle/working/FPTrans/core/metrics.py
import numpy as np


class FewShotMetric(object):
    def __init__(self, n_class):
        self.n_class = n_class
        self.stat = np.zeros((self.n_class + 1, 3))     # +1 for bg, 3 for tp, fp, fn

    def update(self, pred, ref, cls, ori_size=None, verbose=0):
        pred = np.asarray(pred, np.uint8)
        ref = np.asarray(ref, np.uint8)
        for i, ci in enumerate(cls):    # iter on batch ==> [episode_1, episode_2, ...]
            
            p = pred[i]
            r = ref[i]
            if ori_size is not None:
                ori_H, ori_W = ori_size[i]
                p = p[:, :ori_H, :ori_W]
                r = r[:, :ori_H, :ori_W]
           
            # for j, c in enumerate([0, 1 if int(ci) == 255 else int(ci)]):     # iter on class ==> [bg_cls, fg_cls]
            
            # for j, c in enumerate([0, int(ci)]):
            for j, c in enumerate([0, 1]):
                tp = int((np.logical_and(p == j, r != 255) * np.logical_and(r == j, r != 255)).sum())
                fp = int((np.logical_and(p == j, r != 255) * np.logical_and(r != j, r != 255)).sum())
                fn = int((np.logical_and(p != j, r != 255) * np.logical_and(r == j, r != 255)).sum())
                if verbose:
                    print(tp / (tp + fp + fn))
                
                self.stat[c, 0] += tp
                self.stat[c, 1] += fp
                self.stat[c, 2] += fn

    def get_scores(self, labels, binary=False):
        """
        Parameters
        ----------
        labels: list, compute mean iou on which classes
        binary: bool, return mean iou (average on foreground classes) 
            or binary iou (average on foreground and background)

        Returns
        -------
        mIoU_class: np.ndarray, all ious of foreground classes
        mean: float, mean iou over foreground classes
        """
        labels = [0 if lbl == 0 else 1 for lbl in labels]
        if binary:
            stat = np.c_[self.stat[0], self.stat[1:].sum(axis=0)].T     # [2, 3]
        else:
            stat = self.stat[labels]                                    # [N, 3]
        
        tp, fp, fn = stat.T                                             # [2 or N]
        mIoU_class = tp / (tp + fp + fn)                                # [2 or N]
        mean = np.nanmean(mIoU_class)                                   # scalar

        return mIoU_class, mean

    def reset(self):
        self.stat = np.zeros((self.n_class + 1, 3))     # +1 for bg, 3 for tp, fp, fn


class SegmentationMetric(object):
    def __init__(self, n_class):
        self.n_class = n_class
        self.confusion_matrix = np.zeros((n_class, n_class))

    def _fast_hist(self, label_true, label_pred, n_class):
        mask = (label_true >= 0) & (label_true < n_class)
        hist = np.bincount(
            n_class * label_true[mask].astype(int) + label_pred[mask],
            minlength=n_class ** 2,
        ).reshape(n_class, n_class)
        return hist

    def update(self, pred, ref, ori_size=None):
        pred = np.asarray(pred, np.uint8)
        ref = np.asarray(ref, np.uint8)
        for i, (r, p) in enumerate(zip(ref, pred)):
            if ori_size is not None:
                ori_H, ori_W = ori_size[i]
                p = p[:ori_H, :ori_W]
                r = r[:ori_H, :ori_W]
            self.confusion_matrix += self._fast_hist(
                r.flatten(), p.flatten(), self.n_class
            )

    def get_scores(self, binary=True, withbg=True):
        if not binary:
            hist = self.confusion_matrix
        else:
            hist = np.zeros((2, 2), dtype=np.int32)
            hist[0, 0] = self.confusion_matrix[0, 0]
            hist[0, 1] = self.confusion_matrix[0, 1:].sum()
            hist[1, 0] = self.confusion_matrix[1:, 0].sum()
            hist[1, 1] = self.confusion_matrix[1:, 1:].sum()
        iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
        if not withbg:
            iou = iou[1:]
        mean_iou = np.nanmean(iou)

        return iou, mean_iou

    def reset(self):
        self.confusion_matrix = np.zeros((self.n_class, self.n_class))


class Accumulator(object):
    def __init__(self, **kwargs):
        self.values = kwargs
        self.counter = {k: 0 for k, v in kwargs.items()}
        for k, v in self.values.items():
            if not isinstance(v, (float, int, list)):
                raise TypeError(f"The Accumulator does not support `{type(v)}`. Supported types: [float, int, list]")

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(self.values[k], list):
                self.values[k].append(v)
            else:
                self.values[k] = self.values[k] + v
            self.counter[k] += 1

    def reset(self):
        for k in self.values.keys():
            if isinstance(self.values[k], list):
                self.values[k] = []
            else:
                self.values[k] = 0
            self.counter[k] = 0

    def mean(self, key, axis=None, dic=False):
        if isinstance(key, str):
            if isinstance(self.values[key], list):
                return np.array(self.values[key]).mean(axis)
            else:
                return self.values[key] / (self.counter[key] + 1e-7)
        elif isinstance(key, (list, tuple)):
            if dic:
                return {k: self.mean(k, axis) for k in key}
            return [self.mean(k, axis) for k in key]
        else:
            TypeError(f"`key` must be a str/list/tuple, got {type(key)}")

    def std(self, key, axis=None, dic=False):
        if isinstance(key, str):
            if isinstance(self.values[key], list):
                return np.array(self.values[key]).std(axis)
            else:
                raise RuntimeError("`std` is not supported for (int, float). Use list instead.")
        elif isinstance(key, (list, tuple)):
            if dic:
                return {k: self.std(k) for k in key}
            return [self.std(k) for k in key]
        else:
            TypeError(f"`key` must be a str/list/tuple, got {type(key)}")
    def get(self):
        """Returns the current accumulated values."""
        return {k: v.copy() if isinstance(v, list) else v for k, v in self.values.items()}
    
    def load(self, data):
        """Loads accumulated values from a dictionary (used for resuming training)."""
        for k, v in data.items():
            if k in self.values:
                self.values[k] = v.copy() if isinstance(v, list) else v
                self.counter[k] = len(v) if isinstance(v, list) else 0


Overwriting /kaggle/working/FPTrans/core/metrics.py


In [9]:
%%writefile /kaggle/working/FPTrans/core/metrics.py
#NEW CODE 
import numpy as np


class FewShotMetric(object):
    def __init__(self, n_class):
        # Ignore n_class; track binary IoU (defect vs. background)
        self.n_class = n_class  # Kept for compatibility
        self.stat = np.zeros(7)  # [tp, fp, fn , good_catch , positive_pairs , correct_yeild , negative_yeild]

    def update(self, pred, ref, cls, ori_size=None, verbose=0):
        """
        Update metrics for Vision24 defect segmentation (.binary IoU).

        Args:
            pred: Predicted masks [batch, H, W] or [batch, 1, H, W]
            ref: Ground truth masks [batch, H, W], expected to be binary (0, 1)
            cls: Class indices (ignored)
            ori_size: Original sizes [(H, W), ...] (optional)
            verbose: If > 0, print IoU per sample
        """
        pred = np.asarray(pred, np.uint8)
        ref = np.asarray(ref, np.uint8)

        # Validate ground truth: must be binary (0, 1)
        unique_ref = np.unique(ref)
        if not np.all(np.isin(unique_ref, [0, 1])):
            raise ValueError(f"Ground truth masks must be binary (0, 1), got values: {unique_ref}")

        # Convert predictions to binary (threshold > 0)
        pred = (pred > 0).astype(np.uint8)

        for i in range(pred.shape[0]):  # Iterate over batch
            p = pred[i]
            r = ref[i]

            if ori_size is not None:
                ori_H, ori_W = ori_size[i]
                p = p[:ori_H, :ori_W]
                r = r[:ori_H, :ori_W]

            # Binary IoU calculation
            tp = np.sum((p == 1) & (r == 1))  # True positives
            fp = np.sum((p == 1) & (r == 0))  # False positives
            fn = np.sum((p == 0) & (r == 1))  # False negatives

            #iou
            iou = tp / (tp + fp + fn + 1e-7)

            #yeild rate
            if(np.sum((r == 1)) == 0 ):
                #find the all white value in the predict
                defect_in_pred = np.sum((p == 1))
                #all mask area
                all_mask = np.sum((p == 1) & (p == 0))

                #percentage of white space to all space
                percentage  = (defect_in_pred) / (all_mask)
                
                #Adding for the negative pairs , where the mask with no defect 
                self.stat[6] += 1
                
                if(precentage <= 0.000239):
                    self.stat[5] += 1  # good yeild
            else:
                # the number of positive pairs ( where the mask has a defect)
                self.stat[4] +=1
                
                iou = tp / (tp + fp + fn + 1e-7)

                
                if(iou  >= 0.3):
                    #number of good catch where the defect was detected with IOU >= 0.3
                    self.stat[3] +=1 
            
            if verbose:
                #This is a debut if statement
                iou = tp / (tp + fp + fn + 1e-7)         
                print(f"Sample {i} IoU: {iou:.4f}")

            self.stat[0] += tp
            self.stat[1] += fp
            self.stat[2] += fn

    def get_scores(self, labels, binary=False):
        """
        Compute binary IoU.

        Args:
            labels: Ignored, kept for compatibility
            binary: Ignored, always True for Vision24

        Returns:
            mIoU_class: np.ndarray, [iou] (single IoU value)
            mean: float, Same as iou (for compatibility)
        """
        tp, fp, fn ,good_catch,postive_pairs , correct_yeild , negative_pairs = self.stat
        iou = tp / (tp + fp + fn + 1e-7)

        catch_rate=  (good_catch) / (postive_pairs)
        yeild_rate = (correct_yeild) / (negative_pairs)
        
        return np.array([iou]), iou, catch_rate, yeild_rate

    def reset(self):
        self.stat = np.zeros(7)


class SegmentationMetric(object):
    def __init__(self, n_class):
        # Ignore n_class; use 2x2 confusion matrix for binary IoU
        self.n_class = n_class  # Kept for compatibility
        self.confusion_matrix = np.zeros((2, 2))  # [background, foreground]

    def _fast_hist(self, label_true, label_pred):
        # Binary histogram (0, 1)
        mask = (label_true >= 0) & (label_true <= 1)
        hist = np.bincount(
            2 * label_true[mask].astype(int) + label_pred[mask],
            minlength=4,
        ).reshape(2, 2)
        return hist

    def update(self, pred, ref, ori_size=None):
        """
        Update confusion matrix for binary IoU.

        Args:
            pred: Predicted masks [batch, H, W] or [batch, 1, H, W]
            ref: Ground truth masks [batch, H, W], binary (0, 1)
            ori_size: Original sizes [(H, W), ...] (optional)
        """
        pred = np.asarray(pred, np.uint8)
        ref = np.asarray(ref, np.uint8)

        # Validate ground truth
        unique_ref = np.unique(ref)
        if not np.all(np.isin(unique_ref, [0, 1])):
            raise ValueError(f"Ground truth masks must be binary (0, 1), got values: {unique_ref}")

        pred = (pred > 0).astype(np.uint8)

        for i, (r, p) in enumerate(zip(ref, pred)):
            if ori_size is not None:
                ori_H, ori_W = ori_size[i]
                p = p[:ori_H, :ori_W]
                r = r[:ori_H, :ori_W]
            self.confusion_matrix += self._fast_hist(r.flatten(), p.flatten())

    def get_scores(self, binary=False, withbg=True):
        """
        Compute binary IoU from confusion matrix.

        Args:
            binary: Ignored, always True
            withbg: If True, return [bg_iou, fg_iou]; else, [fg_iou]

        Returns:
            iou: np.ndarray, IoU values
            mean_iou: float, Mean IoU
        """
        hist = self.confusion_matrix
        iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist) + 1e-7)
        if not withbg:
            iou = iou[1:]
        mean_iou = np.nanmean(iou)
        return iou, mean_iou

    def reset(self):
        self.confusion_matrix = np.zeros((2, 2))


class Accumulator(object):
    def __init__(self, **kwargs):
        self.values = kwargs
        self.counter = {k: 0 for k, v in kwargs.items()}
        for k, v in self.values.items():
            if not isinstance(v, (float, int, list)):
                raise TypeError(f"The Accumulator does not support `{type(v)}`. Supported types: [float, int, list]")

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(self.values[k], list):
                self.values[k].append(v)
            else:
                self.values[k] = self.values[k] + v
            self.counter[k] += 1

    def reset(self):
        for k in self.values.keys():
            if isinstance(self.values[k], list):
                self.values[k] = []
            else:
                self.values[k] = 0
            self.counter[k] = 0

    def mean(self, key, axis=None, dic=False):
        if isinstance(key, str):
            if isinstance(self.values[key], list):
                return np.array(self.values[key]).mean(axis)
            else:
                return self.values[key] / (self.counter[key] + 1e-7)
        elif isinstance(key, (list, tuple)):
            if dic:
                return {k: self.mean(k, axis) for k in key}
            return [self.mean(k, axis) for k in key]
        else:
            TypeError(f"`key` must be a str/list/tuple, got {type(key)}")

    def std(self, key, axis=None, dic=False):
        if isinstance(key, str):
            if isinstance(self.values[key], list):
                return np.array(self.values[key]).std(axis)
            else:
                raise RuntimeError("`std` is not supported for (int, float). Use list instead.")
        elif isinstance(key, (list, tuple)):
            if dic:
                return {k: self.std(k) for k in key}
            return [self.std(k) for k in key]
        else:
            TypeError(f"`key` must be a str/list/tuple, got {type(key)}")

    def get(self):
        return {k: v.copy() if isinstance(v, list) else v for k, v in self.values.items()}
    
    def load(self, data):
        for k, v in data.items():
            if k in self.values:
                self.values[k] = v.copy() if isinstance(v, list) else v
                self.counter[k] = len(v) if isinstance(v, list) else 0

Overwriting /kaggle/working/FPTrans/core/metrics.py


In [10]:
%%writefile /kaggle/working/FPTrans/data_kits/binary_dataset.py
# %load /kaggle/working/FPTrans/data_kits/binary_dataset.py
import torch
import numpy as np
from torch.utils.data import Dataset
import cv2
import os

class BinarySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_filenames = sorted(os.listdir(image_dir))
        self.mask_filenames = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        # Load image and mask
        image = cv2.imread(image_path)  # BGR image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask in grayscale

        # Convert mask: 255 -> 1 (to match FPTrans format)
        mask = np.where(mask == 255, 1, 0).astype(np.uint8)

        # Apply transformations if provided
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image, mask = transformed["image"], transformed["mask"]

        # Convert to tensors
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        mask = torch.tensor(mask, dtype=torch.long)  # No need to normalize mask

        return image, mask


Writing /kaggle/working/FPTrans/data_kits/binary_dataset.py


In [11]:
%%writefile /kaggle/working/FPTrans/data_kits/tcp_few_shot.py
# %load  /kaggle/working/FPTrans/data_kits/tcp_few_shot.py
from pathlib import Path

import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset

from constants import project_dir, data_dir
from utils_.misc import load_image

cache_image = {}
cache_label = {}


class TCPFewShot(Dataset):
    def __init__(self, opt, split, shot, query,
                 data_root=None, data_list=None, transform=None, mode='train',
                 cache=True):
        assert mode in ['train', 'val', 'test', 'eval_online']

        if mode != "train" and opt.dataset in ["TCP"]:
            mode = "val"
        self.opt = opt
        self.mode = mode
        self.split = split
        self.shot = shot
        self.query = query
        self.data_root = data_root
        self.transform = transform
        self.cache = cache

        self.nclass = 2
        self.data_list = [Path(x) for x in (project_dir / 'data/tcp/tcp.txt').read_text().splitlines()]
        test_range = slice(*opt.test_range)
        self.data_list = self.data_list[test_range]
        self.length_data_list = len(self.data_list)
        self.tasks = list(range(len(self)))
        self.support_image_paths = [list(map(Path, x.split())) for x in (project_dir / 'data/tcp/support.txt').read_text().splitlines()]

        self.sampler = None
        self.reset_sampler()

    def __len__(self):
        return self.opt.test_n or self.length_data_list

    def reset_sampler(self):
        seed = self.opt.seed
        test_seed = self.opt.test_seed
        # Use fixed test sampler(opt.test_seed) for reproducibility
        self.sampler = np.random.RandomState(test_seed)

    def sample_tasks(self):
        pass

    def seg_encode(self, lab, cls, ignore_lab):
        if self.opt.proc == 'pil':
            lab = np.array(lab, np.uint8)
            lab[lab == 255] = 1  # Convert 255 to 1
        if len(lab.shape) == 3:
            lab = lab[:, :, -1]
        assert len(lab.shape) == 2, lab.shape
        target_pix = np.where(lab == cls)
        lab[:, :] = 0
        if target_pix[0].shape[0] > 0:
            lab[target_pix[0], target_pix[1]] = 1
        if self.opt.proc == 'pil':
            lab = Image.fromarray(lab)
        return lab

    def get_image(self, name, cache=True):
        if self.cache and cache:
            if name not in cache_image:
                cache_image[name] = load_image(data_dir / name, 'img', self.opt.proc)
            return cache_image[name].copy()
        else:
            return load_image(data_dir / name, 'img', self.opt.proc)

    def get_label(self, name, cls, ignore_lab=0, cache=True):
        if self.cache and cache:
            if name not in cache_label:
                cache_label[name] = load_image(data_dir / name, 'lab', self.opt.proc)
            lab = cache_label[name].copy()
        else:
            lab = load_image(data_dir / name, 'lab', self.opt.proc)
        lab = self.seg_encode(lab, cls, ignore_lab)
        lab[lab == 255] = 1  # Convert 255 to 1
        return lab

    def __getitem__(self, index):
        qry_idx = self.tasks[index]
        image_path = self.data_list[qry_idx]
        sup_image_paths = []
        sup_label_paths = []
        for x_path, y_path in self.support_image_paths:
            sup_image_paths.append(x_path)
            sup_label_paths.append(y_path)

        qry_names = [image_path.stem]
        sup_names = [x.stem for x in sup_image_paths]

        ori_image = image = self.get_image(image_path)
        label = np.zeros(image.shape[:2], dtype=np.uint8)
        ori_image = torch.from_numpy(ori_image)

        kwargs = {}
        sup_kwargs = [{} for _ in range(self.opt.shot)]
        sup_images = [self.get_image(x) for x in sup_image_paths]
        sup_labels = [self.get_label(x, 255) for x in sup_label_paths]

        # raw_label = label.copy()
        if self.transform is not None:
            image, label, kwargs = self.transform[1](image, label, **kwargs)
            for k in range(self.shot):
                sup_images[k], sup_labels[k], sup_kwargs[k] = self.transform[0](sup_images[k], sup_labels[k],
                                                                                **sup_kwargs[k])

        sup_images = torch.stack(sup_images, dim=0)
        sup_labels = torch.stack(sup_labels, dim=0)

        ret_dict = {
            'sup_rgb': sup_images,  # [S, 3, H, W]
            'sup_msk': sup_labels,  # [S, H, W]
            'qry_rgb': image,       # [3, H, W]
            'qry_ori': ori_image,   # [H_ori, W_ori, 3]
            'cls': 255,  # [], values in [1, 20] for PASCAL
            'sup_names': sup_names,
            'qry_names': qry_names,
        }

        ret_dict = {k: v for k, v in ret_dict.items() if v is not None}
        return ret_dict


Overwriting /kaggle/working/FPTrans/data_kits/tcp_few_shot.py


In [12]:
%%writefile /kaggle/working/FPTrans/data_kits/transformation.py
# %load /kaggle/working/FPTrans/data_kits/transformation.py
"""
A transformation package for image segmentation. It is implemented by PIL and
torchvision and its usage is the same as torchvision, except for all the
classes accept (img, mask) as input and output (img, mask).

Notes:
Opencv(cv2)-processed images and PIL-processed images may have some differences,
which may lead to incompatibility between the images and pretrained models and
performance drop.
"""

import torchvision.transforms as transforms
import torchvision.transforms.functional as tf
import numpy as np
from PIL import Image, ImageFilter
import random
import torch
import cv2

__all__ = [
    "Compose",
    "Resize", "ToTensor", "Normalize",
    "ColorJitter", "RandomCrop", "RandomResize", "RandomSizedCrop",
    "RandomHorizontallyFlip", "RandomVerticallyFlip", "RandomRotate",
    "RandomGaussianBlur"
]
RGB_MEAN = np.array([0.485, 0.456, 0.406]) * 255


class Compose(object):
    def __init__(self, transformations, processer='pil'):
        self.transforms = transformations
        self.processer = processer

    def __call__(self, image, label, **kwargs):
        if self.processer == 'cv2':
            image = np.float32(image)
            label[label == 255] = 1  # Convert 255 to 1
            label = np.uint8(label)
        for t in self.transforms:
            image, label, kwargs = t(image, label, proc=self.processer, **kwargs)
            
        return image, label, kwargs


###############################################################################
#
#   Deterministic Algorithms
#
###############################################################################

class Resize(object):
    """
    Resize a pair of (image, mask).
    If `height` and `width` are both specified, then the aspect ratio is not
    keeped. If only one of the `height` and `width` is specified, the aspect
    ratio will be preserved.

    Parameters
    ----------
    height: int
    width: int
    do_mask: bool
        Do resize on mask or not.
    """
    def __init__(self, height=None, width=None, do_mask=True):
        if height is None and width is None:
            raise ValueError('At least specify one of the `height` and '
                             '`width`')
        self.height = height
        self.width = width
        self.do_mask = do_mask

    def __call__(self, img, mask, proc='pil', **kwargs):
        if proc == 'pil':
            w, h = img.size
        else:
            h, w, _ = img.shape
        ow, oh = self.width, self.height
        if self.height is None:
            oh = int(self.width * h / w)
        if self.width is None:
            ow = int(self.height * w / h)

        if proc == 'pil':
            img = img.resize((ow, oh), Image.BILINEAR)
            if self.do_mask:
                mask = mask.resize((ow, oh), Image.NEAREST)
        else:
            img = cv2.resize(img, (ow, oh), interpolation=cv2.INTER_LINEAR)
            if self.do_mask:
                mask = cv2.resize(mask, (ow, oh), interpolation=cv2.INTER_NEAREST)

        if 'weights' in kwargs:
            kwargs['weights'] = cv2.resize(kwargs['weights'], (ow, oh), interpolation=cv2.INTER_LINEAR)

        return img, mask, kwargs


class ToTensor(object):
    def __init__(self, mask_dtype):
        self.mask_dtype = mask_dtype

    def __call__(self, img, mask, proc='pil', **kwargs):
        img = torch.from_numpy(
            np.ascontiguousarray(np.asarray(img).transpose(2, 0, 1))
        ).float().div(255)

        mask = torch.from_numpy(np.array(mask, np.uint8))
        mask[mask == 255] = 1  # Convert 255 to 1
        if self.mask_dtype == 'long':
            mask = mask.long()
        elif self.mask_dtype == 'float':
            mask = mask.float()

        if 'weights' in kwargs:
            kwargs['weights'] = torch.from_numpy(kwargs['weights']).float()

        return img, mask, kwargs


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, img, mask, proc='pil', **kwargs):
        img = tf.normalize(img, self.mean, self.std)

        return img, mask, kwargs


###############################################################################
#
#   Random Algorithms
#
###############################################################################

class ColorJitter(object):
    def __init__(self, brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.5):
        self.p = p
        self.jitter = transforms.ColorJitter(
            brightness, contrast, saturation, hue
        )

    def __call__(self, img, mask, proc='pil', **kwargs):
        if random.random() > self.p:
            if proc == 'pil':
                img = self.jitter(img)
            else:
                img = Image.fromarray(img.astype(np.uint8))
                img = self.jitter(img)
                img = np.array(img)

        return img, mask, kwargs


class RandomGaussianBlur(object):
    def __init__(self, radius=5, rand_radius=False):
        self.radius = radius
        self.rand_radius = rand_radius

    def __call__(self, img, mask, proc='pil', **kwargs):
        if random.random() < 0.5:
            if proc == 'pil':
                img = img.filter(ImageFilter.GaussianBlur(self.radius))
            else:
                radius = random.choice([self.radius, self.radius - 2, self.radius + 2]) if self.rand_radius else self.radius
                img = cv2.GaussianBlur(img, (radius, radius), 0)

        return img, mask, kwargs


class RandomCrop(object):
    """
    Random crop a patch from the (img, mask) pair.

    Parameters
    ----------
    height: int
        Output height
    width: int
        Output width
    check: bool
        Whether to check the cropped mask contains a minimum proportion of
        original foreground/background pixels. Only support two classes
    prop: float
        Foreground pixel proportion.
    max_iter: int
        Maximum iterations to try (to meet the `check` requirement).
    """
    def __init__(self, height, width, check=False, prop=0.85, max_iter=30, center=True, pad_type='reflect'):
        self.height = height
        self.width = width
        self.check = check
        self.prop = prop
        self.max_iter = max_iter
        self.center = center
        if pad_type == 'reflect':
            self.pad_kwargs_image = self.pad_kwargs_mask = self.pad_kwargs_weight = {
                'borderType': cv2.BORDER_REFLECT101
            }
        elif pad_type == 'constant':
            self.pad_kwargs_image = {
                'borderType': cv2.BORDER_CONSTANT,
                'value': RGB_MEAN
            }
            self.pad_kwargs_mask = {
                'borderType': cv2.BORDER_CONSTANT,
                'value': 255,
            }
            self.pad_kwargs_weight = {
                'borderType': cv2.BORDER_CONSTANT,
                'value': 0,
            }
        else:
            raise ValueError

    def __call__(self, img, mask, proc='pil', **kwargs):
        if proc == 'pil':
            w, h = img.size
            mw, mh = mask.size
        else:
            h, w, _ = img.shape
            mh, mw = mask.shape
        tw, th = self.width, self.height
        assert w == mw and h == mh, f'(W, H) => ({w}, {h}) vs ({mw}, {mh})'
        weights = kwargs.get('weights', None)

        if proc not in ['pil', 'cv2']:
            raise ValueError

        if w != tw or h != th:
            if w < tw or h < th:
                if proc == 'pil':
                    img = img.resize((tw, th), Image.BILINEAR)
                    mask = mask.resize((tw, th), Image.NEAREST)
                else:
                    # img = cv2.resize(img, (tw, th), interpolation=cv2.INTER_LINEAR)
                    # mask = cv2.resize(mask, (tw, th), interpolation=cv2.INTER_NEAREST)
                    pad_h = max(th - h, 0)
                    pad_w = max(tw - w, 0)
                    if self.center:
                        pad_h_half = int(pad_h / 2)
                        pad_w_half = int(pad_w / 2)
                    else:
                        pad_h_half = random.randint(0, pad_h)
                        pad_w_half = random.randint(0, pad_w)
                    borders = (pad_h_half, pad_h - pad_h_half, pad_w_half, pad_w - pad_w_half)
                    if pad_h > 0 or pad_w > 0:
                        img = cv2.copyMakeBorder(img, *borders, **self.pad_kwargs_image)
                        mask = cv2.copyMakeBorder(mask, *borders, **self.pad_kwargs_mask)
                if weights is not None:
                    weights = cv2.copyMakeBorder(weights, *borders, **self.pad_kwargs_weight)

            raw_img = img
            raw_mask = mask
            raw_weights = weights
            if proc == 'pil':
                w, h = img.size
            else:
                h, w, _ = img.shape
            y1 = random.randint(0, h - th)
            x1 = random.randint(0, w - tw)
            if proc == 'pil':
                img = raw_img.crop((x1, y1, x1 + tw, y1 + th))
                mask = raw_mask.crop((x1, y1, x1 + tw, y1 + th))
            else:
                img = raw_img[y1:y1 + th, x1:x1 + tw]
                mask = raw_mask[y1:y1 + th, x1:x1 + tw]

            if weights is not None:
                weights = weights[y1:y1 + th, x1:x1 + tw]

            if self.check:
                # For semgentation, we must assert the cropped patch contains
                # foreground pixels
                np_raw_mask = np.array(raw_mask, np.uint8)
                np_mask = np.array(mask, np.uint8)
                np_mask[np_mask == 255] = 1  # Convert 255 to 1
                raw_pos_num = np.sum(np_raw_mask == 1)
                pos_num = np.sum(np_mask == 1)
                crop_iter = 0
                while pos_num < self.prop * raw_pos_num and crop_iter <= self.max_iter:
                    image = raw_img
                    label = raw_mask
                    weights = raw_weights
                    y1 = random.randint(0, h - th)
                    x1 = random.randint(0, w - tw)
                    if proc == 'pil':
                        img = raw_img.crop((x1, y1, x1 + tw, y1 + th))
                        mask = raw_mask.crop((x1, y1, x1 + tw, y1 + th))
                    else:
                        img = raw_img[y1:y1 + th, x1:x1 + tw]
                        mask = raw_mask[y1:y1 + th, x1:x1 + tw]
                    if weights is not None:
                        weights = weights[y1:y1 + th, x1:x1 + tw]
                    crop_iter += 1
                if crop_iter >= self.max_iter:
                    if proc == 'pil':
                        img = raw_img.resize((tw, th), Image.BILINEAR)
                        mask = raw_mask.resize((tw, th), Image.NEAREST)
                    else:
                        img = cv2.resize(img, (tw, th), interpolation=cv2.INTER_LINEAR)
                        mask = cv2.resize(mask, (tw, th), interpolation=cv2.INTER_NEAREST)
                    if weights is not None:
                        weights = cv2.resize(weights, (tw, th), interpolation=cv2.INTER_LINEAR)

        if weights is not None:
            kwargs['weights'] = weights

        return img, mask, kwargs


class RandomResize(object):
    def __init__(self, smin, smax):
        self.smin = smin
        self.smax = smax

    def __call__(self, img, mask, proc='pil', **kwargs):
        if proc == 'pil':
            w, h = img.size
            mw, mh = mask.size
        else:
            h, w, _ = img.shape
            mh, mw = mask.shape
        assert w == mw and h == mh, f'(W, H) => ({w}, {h}) vs ({mw}, {mh})'

        scale = self.smin + (self.smax - self.smin) * random.random()

        tw = int(w * scale)
        th = int(h * scale)

        if proc == 'pil':
            img = img.resize((tw, th), Image.BILINEAR)
            mask = mask.resize((tw, th), Image.NEAREST)
        else:
            img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
            mask = cv2.resize(mask, None, fx=scale, fy=scale, interpolation=cv2.INTER_NEAREST)

        if 'weights' in kwargs:
            kwargs['weights'] = cv2.resize(kwargs['weights'], None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

        return img, mask, kwargs


class RandomSizedCrop(object):
    def __init__(self, smin, smax, height, width,
                 check=False, fg_prop=0.85, bg_prop=0.15, max_iter=30, center=True, pad_type='reflect'):
        self.resize = RandomResize(smin, smax)
        self.crop = RandomCrop(height, width, check, fg_prop, bg_prop, max_iter, center, pad_type)

    def __call__(self, img, mask, proc='pil', **kwargs):
        img, mask, kwargs = self.resize(img, mask, proc=proc, **kwargs)
        img, mask, kwargs = self.crop(img, mask, proc=proc, **kwargs)
        return img, mask, kwargs


class RandomHorizontallyFlip(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, img, mask, proc='pil', **kwargs):
        if random.random() < self.p:
            if proc == 'pil':
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
                mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
            else:
                img = np.ascontiguousarray(np.flip(img, axis=1))
                mask = np.ascontiguousarray(np.flip(mask, axis=1))
            if 'weights' in kwargs:
                kwargs['weights'] = np.ascontiguousarray(np.flip(kwargs['weights'], axis=1))

        return img, mask, kwargs


class RandomVerticallyFlip(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img, mask, proc='pil', **kwargs):
        if random.random() < self.p:
            if proc == 'pil':
                img = img.transpose(Image.FLIP_TOP_BOTTOM)
                mask = mask.transpose(Image.FLIP_TOP_BOTTOM)
            else:
                img = np.ascontiguousarray(np.flip(img, axis=0))
                mask = np.ascontiguousarray(np.flip(mask, axis=0))
            if 'weights' in kwargs:
                kwargs['weights'] = np.ascontiguousarray(np.flip(kwargs['weights'], axis=0))

        return img, mask, kwargs


class RandomRotate(object):
    def __init__(self, rotate, p=0.5, pad_type='reflect'):
        self.rotate = rotate
        self.p = p
        if pad_type == 'reflect':
            self.pad_kwargs_image = self.pad_kwargs_mask = self.pad_kwargs_weight = {
                'borderMode': cv2.BORDER_REFLECT101
            }
        elif pad_type == 'constant':
            self.pad_kwargs_image = {
                'borderMode': cv2.BORDER_CONSTANT,
                'borderValue': RGB_MEAN,
            }
            self.pad_kwargs_mask = {
                'borderMode': cv2.BORDER_CONSTANT,
                'borderValue': 255,
            }
            self.pad_kwargs_weight = {
                'borderMode': cv2.BORDER_CONSTANT,
                'borderValue': 0,
            }
        else:
            raise ValueError

    def __call__(self, img, mask, proc='pil', **kwargs):
        if random.random() < self.p:
            angle = -self.rotate + 2 * self.rotate * random.random()

            if proc == 'pil':
                img, mask = np.array(img, np.uint8), np.array(mask, np.uint8)

            h, w, _ = img.shape
            matrix = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
            img = cv2.warpAffine(img, matrix, (w, h), flags=cv2.INTER_LINEAR, **self.pad_kwargs_image)
            mask = cv2.warpAffine(mask, matrix, (w, h), flags=cv2.INTER_NEAREST, **self.pad_kwargs_mask)
            if proc == 'pil':
                img, mask = Image.fromarray(img), Image.fromarray(mask)

            if 'weights' in kwargs:
                kwargs['weights'] = cv2.warpAffine(kwargs['weights'], matrix, (w, h), flags=cv2.INTER_LINEAR, **self.pad_kwargs_weight)
        
        return img, mask, kwargs



Overwriting /kaggle/working/FPTrans/data_kits/transformation.py


In [13]:
%%writefile /kaggle/working/FPTrans/run.py
# %load /kaggle/working/FPTrans/run.py
from pathlib import Path

import numpy as np
import torch
from PIL import Image
from sacred import Experiment

from config import setup, init_environment
from constants import on_cloud
from core.base_trainer import BaseTrainer, BaseEvaluator
from core.losses import get as get_loss_obj
from data_kits import datasets
from networks import load_model
from utils_ import misc

ex = setup(
    Experiment(name="FPTrans", save_git_info=False, base_dir="./")
)
torch.set_printoptions(precision=8)

class Evaluator(BaseEvaluator):
    def test_step(self, batch, step):
        sup_rgb = batch['sup_rgb'].cuda()
        sup_msk = batch['sup_msk'].cuda()
        qry_rgb = batch['qry_rgb'].cuda()
        qry_msk = batch['qry_msk'].cuda()
        classes = batch['cls'].cuda()

        output = self.model_DP(qry_rgb, sup_rgb, sup_msk, qry_msk)
        qry_pred = output['out']

        # Compute loss
        loss = self.loss_obj(qry_pred, qry_msk.squeeze(1))

        # Compute prediction
        qry_pred = qry_pred.argmax(dim=1).detach().cpu().numpy()
        return qry_pred, {'loss': loss.item()}


class Trainer(BaseTrainer):
    def _train_step(self, batch, step, epoch):
        sup_rgb = batch['sup_rgb'].cuda()
        sup_msk = batch['sup_msk'].cuda()
        qry_rgb = batch['qry_rgb'].cuda()
        qry_msk = batch['qry_msk'].cuda()
        classes = batch['cls'].cuda()
        kwargs = {}
        if 'weights' in batch:
            kwargs['weight'] = batch['weights'].cuda()

        output = self.model_DP(qry_rgb, sup_rgb, sup_msk, qry_msk)
        qry_msk_reshape = qry_msk.view(-1, *qry_msk.shape[-2:])

        loss = self.loss_obj(output['out'], qry_msk_reshape, **kwargs)        
        loss_prompt = self.loss_obj(output['out_prompt'], qry_msk_reshape, **kwargs)
        if len(output['loss_pair'].shape) == 0:     # single GPU
            loss_pair = output['loss_pair']
        else:   # multiple GPUs
            loss_pair = output['loss_pair'].mean(0)
        loss_pair = loss_pair * self.opt.pair_lossW
 
        total_loss = loss + loss_prompt + loss_pair
        return total_loss, loss, loss_prompt, loss_pair

    def train_step(self, batch, step, epoch):
        self.optimizer.zero_grad()

        total_loss, loss, loss_prompt, loss_pair = self._train_step(batch, step, epoch)

        total_loss.backward()
        self.optimizer.step()
        
        return {
            'loss': loss.item(),
            'prompt': loss_prompt.item(),
            'pair': loss_pair.item(),
        }


@ex.main
def train(_run, _config):
    opt, logger, device = init_environment(ex, _run, _config)

    ds_train, data_loader, _ = datasets.load(opt, logger, "train")
    ds_eval_online, data_loader_val, num_classes = datasets.load(opt, logger, "eval_online")
    
    logger.info(f'     ==> {len(ds_train)} training samples')
    logger.info(f'     ==> {len(ds_eval_online)} eval_online samples')

    model = load_model(opt, logger)
    if opt.exp_id >= 0 or opt.ckpt:
        
        ckpt = misc.find_snapshot(_run.run_dir.parent, opt.exp_id, opt.ckpt, afs=on_cloud)
        model.load_weights(ckpt, logger, strict=opt.strict)

    trainer = Trainer(opt, logger, device, model, data_loader, data_loader_val, _run)
    evaluator = Evaluator(opt, logger, device, trainer.model_DP, None, "EVAL_ONLINE")

    logger.info("Start training.")
    start_epoch = 1
    trainer.start_training_loop(start_epoch, evaluator, num_classes)

    logger.info(f"============ Training finished - id {_run._id} ============\n")
    if _run._id is not None:
        return test(_run, _config, _run._id, ckpt=None, strict=False, eval_after_train=True)


@ex.command(unobserved=True)
def test(_run, _config, exp_id=-1, ckpt=None, strict=True, eval_after_train=False):
    opt, logger, device = init_environment(ex, _run, _config, eval_after_train=eval_after_train)

    ds_test, data_loader, num_classes = datasets.load(opt, logger, "test")
    
    logger.info(f'     ==> {len(ds_test)} testing samples')

    model = load_model(opt, logger)
    if not opt.no_resume:
        model_ckpt = misc.find_snapshot(_run.run_dir.parent, exp_id, ckpt)
        logger.info(f"     ==> Try to load checkpoint from {model_ckpt}")
        model.load_weights(model_ckpt, logger, strict=strict)
        logger.info(f"     ==> Checkpoint loaded.")

    tester = Evaluator(opt, logger, device, model, None, "EVAL")

    logger.info("Start testing.")
    loss, mean_iou, binary_iou, _, _ , catch_rate , yeild_rate = tester.start_eval_loop(data_loader, num_classes)

    return f"Loss: {loss:.4f}, mIoU: {mean_iou * 100:.2f}, bIoU: {binary_iou * 100:.2f} , catch_rate:{catch_rate * 100:.2f}, yeild_rate: {yeild_rate* 100:.2f}"


@ex.command(unobserved=True)
def predict(_run, _config, exp_id=-1, ckpt=None, strict=True):
    opt, logger, device = init_environment(ex, _run, _config)

    model = load_model(opt, logger)
    if not opt.no_resume:
        model_ckpt = misc.find_snapshot(_run.run_dir.parent, exp_id, ckpt)
        logger.info(f"     ==> Try to load checkpoint from {model_ckpt}")
        model.load_weights(model_ckpt, logger, strict)
        logger.info(f"     ==> Checkpoint loaded.")
    model = model.to(device)
    loss_obj = get_loss_obj(opt, logger, loss='ce')

    sup_rgb, sup_msk, qry_rgb, qry_msk, qry_ori = datasets.load_p(opt, device)
    classes = torch.LongTensor([opt.p.cls]).cuda()

    logger.info("Start predicting.")

    model.eval()
    ret_values = []
    for i in range(qry_rgb.shape[0]):
        print('Processing:', i + 1)
        qry_rgb_i = qry_rgb[i:i + 1]
        qry_msk_i = qry_msk[i:i + 1] if qry_msk is not None else None
        qry_ori_i = qry_ori[i]
        
        output = model(qry_rgb_i, sup_rgb, sup_msk, out_shape=qry_ori_i.shape[-3:-1])
        pred = output['out'].argmax(dim=1).detach().cpu().numpy()

        if qry_msk_i is not None:
            loss = loss_obj(output['out'], qry_msk_i).item()
            ref = qry_msk_i.cpu().numpy()
            tp = int((np.logical_and(pred == 1, ref != 255) * np.logical_and(ref == 1, ref != 255)).sum())
            fp = int((np.logical_and(pred == 1, ref != 255) * np.logical_and(ref != 1, ref != 255)).sum())
            fn = int((np.logical_and(pred != 1, ref != 255) * np.logical_and(ref == 1, ref != 255)).sum())
            mean_iou = tp / (tp + fp + fn)
            binary_iou = 0
            ret_values.append(f"Loss: {loss:.4f}, mIoU: {mean_iou * 100:.2f}, bIoU: {binary_iou * 100:.2f}")
        else:
            ret_values.append(None)

        # Save to file
        if opt.p.out:
            pred = pred[0].astype(np.uint8) * 255
            if opt.p.overlap:
                out = qry_ori_i.copy()
                # out[pred == 255] = out[pred == 255] * 0.5 + np.array([255, 0, 0]) * 0.5
                pred[pred == 255] = 1  # Ensure consistency
                out[pred == 1] = out[pred == 1] * 0.5 + np.array([255, 0, 0]) * 0.5
            else:
                out = pred

            out_dir = Path(opt.p.out)
            out_dir.mkdir(parents=True, exist_ok=True)
            out_name = Path(opt.p.qry or opt.p.qry_rgb[i]).stem + '_pred.png'
            out_path = out_dir / out_name
            Image.fromarray(out).save(out_path)

        # Release memory
        del output
        torch.cuda.empty_cache()

    if ret_values[0] is not None:
        return '\n'.join(ret_values)


if __name__ == '__main__':
    ex.run_commandline()


Overwriting /kaggle/working/FPTrans/run.py


In [14]:
%%writefile /kaggle/working/FPTrans/networks/vit.py
# %load /kaggle/working/FPTrans/networks/vit.py
"""Vision Transformer (ViT) in PyTorch
A PyTorch implement of Vision Transformers as described in:
'An Image Is Worth 16 x 16 Words: Transformers for Image Recognition at Scale'
    - https://arxiv.org/abs/2010.11929
`How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers`
    - https://arxiv.org/abs/2106.10270
The official jax code is released and available at https://github.com/google-research/vision_transformer
DeiT model defs and weights from https://github.com/facebookresearch/deit,
paper `DeiT: Data-efficient Image Transformers` - https://arxiv.org/abs/2012.12877
Acknowledgments:
* The paper authors for releasing code and weights, thanks!
* I fixed my class token impl based on Phil Wang's https://github.com/lucidrains/vit-pytorch ... check it out
for some einops/einsum fun
* Simple transformer style inspired by Andrej Karpathy's https://github.com/karpathy/minGPT
* Bert reference code checks against Huggingface Transformers and Tensorflow Bert
Hacked together by / Copyright 2021 Ross Wightman

# ===========================================================================================
# Feature-Proxy Transformer (FPTrans) in PyTorch
#
# This code file is copied and modified from
#     https://github.com/lucidrains/vit-pytorch/blob/main/vit_pytorch/vit.py
# for the development of FPTrans.
#
# By Jian-Wei Zhang (zjw.math@qq.com).
# 2022-09
#
# ===========================================================================================

"""
import math
import logging
from functools import partial
from collections import OrderedDict
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from networks import vit_utils as utils

_logger = logging.getLogger(name=Path(__file__).parents[1].stem)

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)
IMAGENET_INCEPTION_MEAN = (0.5, 0.5, 0.5)
IMAGENET_INCEPTION_STD = (0.5, 0.5, 0.5)


def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic', 'fixed_input_size': True,
        'mean': IMAGENET_INCEPTION_MEAN, 'std': IMAGENET_INCEPTION_STD,
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


default_cfgs = {
    # patch models (weights from official Google JAX impl)
    'vit_tiny_patch16_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'),
    'vit_tiny_patch16_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_small_patch32_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'),
    'vit_small_patch32_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_small_patch16_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'),
    'vit_small_patch16_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_base_patch32_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'),
    'vit_base_patch32_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_32-i21k-300ep-lr_0.001-aug_light1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_base_patch16_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_224.npz'),
    'vit_base_patch16_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_base_patch8_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_8-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_224.npz'),
    'vit_large_patch32_224': _cfg(
        url='',  # no official model weights for this combo, only for in21k
    ),
    'vit_large_patch32_384': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_p32_384-9b920ba8.pth',
        input_size=(3, 384, 384), crop_pct=1.0),
    'vit_large_patch16_224': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz'),
    'vit_large_patch16_384': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_384.npz',
        input_size=(3, 384, 384), crop_pct=1.0),

    # patch models, imagenet21k (weights from official Google JAX impl)
    'vit_tiny_patch16_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_small_patch32_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_small_patch16_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_base_patch32_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_base_patch16_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_base_patch8_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/B_8-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0.npz',
        num_classes=21843),
    'vit_large_patch32_224_in21k': _cfg(
        url='https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_large_patch32_224_in21k-9046d2e7.pth',
        num_classes=21843),
    'vit_large_patch16_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/augreg/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1.npz',
        num_classes=21843),
    'vit_huge_patch14_224_in21k': _cfg(
        url='https://storage.googleapis.com/vit_models/imagenet21k/ViT-H_14.npz',
        hf_hub='timm/vit_huge_patch14_224_in21k',
        num_classes=21843),

    # SAM trained models (https://arxiv.org/abs/2106.01548)
    'vit_base_patch32_sam_224': _cfg(
        url='https://storage.googleapis.com/vit_models/sam/ViT-B_32.npz'),
    'vit_base_patch16_sam_224': _cfg(
        url='https://storage.googleapis.com/vit_models/sam/ViT-B_16.npz'),

    # deit models (FB weights)
    'deit_tiny_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    'deit_small_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_small_patch16_224-cd65a155.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    'deit_base_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    'deit_base_patch16_384': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_384-8de9b5d1.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, input_size=(3, 384, 384), crop_pct=1.0),
    'deit_tiny_distilled_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_tiny_distilled_patch16_224-b40b3cf7.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, classifier=('head', 'head_dist')),
    'deit_small_distilled_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_small_distilled_patch16_224-649709d9.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, classifier=('head', 'head_dist')),
    'deit_base_distilled_patch16_224': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_base_distilled_patch16_224-df68dfff.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, classifier=('head', 'head_dist')),
    'deit_base_distilled_patch16_384': _cfg(
        url='https://dl.fbaipublicfiles.com/deit/deit_base_distilled_patch16_384-d0272ac0.pth',
        mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, input_size=(3, 384, 384), crop_pct=1.0,
        classifier=('head', 'head_dist')),

    # ViT ImageNet-21K-P pretraining by MILL
    'vit_base_patch16_224_miil_in21k': _cfg(
        url='https://miil-public-eu.oss-eu-central-1.aliyuncs.com/model-zoo/ImageNet_21K_P/models/timm/vit_base_patch16_224_in21k_miil.pth',
        mean=(0, 0, 0), std=(1, 1, 1), crop_pct=0.875, interpolation='bilinear', num_classes=11221,
    ),
    'vit_base_patch16_224_miil': _cfg(
        url='https://miil-public-eu.oss-eu-central-1.aliyuncs.com/model-zoo/ImageNet_21K_P/models/timm'
            '/vit_base_patch16_224_1k_miil_84_4.pth',
        mean=(0, 0, 0), std=(1, 1, 1), crop_pct=0.875, interpolation='bilinear',
    ),
}


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)  # make torchscript happy (cannot use tensor as tuple)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = utils.DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = utils.Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class VisionTransformer(nn.Module):
    """ Vision Transformer
    A PyTorch impl of : `An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale`
        - https://arxiv.org/abs/2010.11929
    Includes distillation token & head support for `DeiT: Data-efficient Image Transformers`
        - https://arxiv.org/abs/2012.12877
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
                 pretrained="",
                 num_heads=12, mlp_ratio=4., qkv_bias=True, representation_size=None, distilled=False,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0., embed_layer=utils.PatchEmbed, norm_layer=None,
                 act_layer=None, weight_init='',
                 opt=None, logger=None, original=False):
        """
        Args:
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_chans (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (int): embedding dimension
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            distilled (bool): model includes a distillation token and head as in DeiT models
            drop_rate (float): dropout rate
            attn_drop_rate (float): attention dropout rate
            drop_path_rate (float): stochastic depth rate
            embed_layer (nn.Module): patch embedding layer
            norm_layer: (nn.Module): normalization layer
            weight_init: (str): weight init scheme
        """
        super().__init__()
        self.opt = opt
        self.logger = logger
        self.allow_mod = not original
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_tokens = 2 if distilled else 1
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU

        # Patch embedding
        self.patch_embed = embed_layer(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim, stride=opt.vit_stride)
        num_patches = self.patch_embed.num_patches

        # prompt tokens
        self.prompt_tokens = None
        if self.allow_mod:
            ncls = 15 if opt.dataset == "PASCAL" else 60
            if opt.coco2pascal:
                ncls = 60
            divider = 1 + opt.bg_num * opt.shot
            self.prompt_tokens = nn.Parameter(
                torch.zeros(ncls * divider, opt.num_prompt // divider, embed_dim))  # [bank_size, G, embed_dim]
            nn.init.normal_(self.prompt_tokens.permute(2, 0, 1), std=opt.pt_std)
            self.sampler = np.random.RandomState(1234)

        # Class token, Distillation token
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.blocks = nn.Sequential(*[
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, drop=drop_rate,
                attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ('fc', nn.Linear(embed_dim, representation_size)),
                ('act', nn.Tanh())
            ]))
        else:
            self.pre_logits = nn.Identity()

        # Classifier head(s)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()
        if pretrained == "":
            self.init_weights(weight_init)
        else:
            if str(pretrained).endswith('.pth'):
                _load_weights_pth(logger, self, pretrained)
            elif str(pretrained).endswith('.npz'):
                _load_weights_npz(self, pretrained)
            else:
                raise ValueError(f'Not recognized file {pretrained}. [.pth|.npz]')

            if logger is not None:
                logger.info(' ' * 5 + f'==> {opt.backbone} initialized from {pretrained}')

    def init_weights(self, mode=''):
        assert mode in ('jax', 'jax_nlhb', 'nlhb', '')
        head_bias = -math.log(self.num_classes) if 'nlhb' in mode else 0.
        utils.trunc_normal_(self.pos_embed, std=.02)
        if self.dist_token is not None:
            utils.trunc_normal_(self.dist_token, std=.02)
        if mode.startswith('jax'):
            # leave cls token as zeros to match jax impl
            utils.named_apply(partial(_init_vit_weights, head_bias=head_bias, jax_impl=True), self)
        else:
            utils.trunc_normal_(self.cls_token, std=.02)
            self.apply(_init_vit_weights)

    def forward_original(self, x):
        # x: [B, C, H, W]
        x = self.patch_embed(x)  # [B, N ,C]
        cls_token = expand_to_batch(self.cls_token, x.shape[0])  # [B, 1, C]
        if self.dist_token is None:
            x = cat_token(cls_token, x)     # [B, N+1, C]
        else:
            dist_token = expand_to_batch(self.dist_token, x.shape[0])  # [B, 1, C]
            x = cat_token(cls_token, dist_token, x)     # [B, N+2, C]
        x = x + self.pos_embed  # [B, N + 1, C]
        x = self.pos_drop(x)

        for i, block in enumerate(self.blocks):
            x = block(x)
        x = self.norm(x)

        x = x[:, self.num_tokens:, :]
        b, n, c = x.shape
        hh = int(math.sqrt(n))
        x = x.view(b, hh, hh, c).permute(0, 3, 1, 2).contiguous()

        return dict(out=x)

    def forward(self, x):
        if not self.allow_mod:
            return self.forward_original(x)

        # x: [B, C, H, W]
        # fg_token: [B, 1, C]
        # bg_token: [B, k, C]
        x, (fg_token, bg_token) = x
        bank_size, G, embed_dim = self.prompt_tokens.shape
        B = x.shape[0] // (1 + self.opt.shot)
        divider = 1 + self.opt.bg_num * self.opt.shot
        prompts = self.prompt_tokens[self.sampler.choice(bank_size, size=B * divider, replace=False)] \
            .reshape(B, divider * G, embed_dim)    # [B, (1+bg_num*shot)*G, embed_dim]

        # print("bg_token shape:", bg_token.shape)
        # print("prompts shape:", prompts.shape)
        # print("G:", G)
        # print("reshaping to:", bg_token.unsqueeze(2).expand(-1, -1, G, -1).shape)
        value = ""
        if(self.opt.dataset == "VISION24"):
            bg_token = bg_token[:, 0:1, :]  # shape: [1, 1, 768]
            bg = bg_token.unsqueeze(2)                    # [1, 1, 1, 768]
            bg = bg.expand(B, (divider - 1), G, embed_dim)  # [3, 1, 12, 768]
            bg = bg.reshape(B, (divider - 1) * G, embed_dim)  # [3, 12, 768]
            value = prompts[:, G:] + bg

        else:       
            value = prompts[:, G:] + bg_token.unsqueeze(2).expand(-1, -1, G, -1).reshape(
                    B, (divider - 1) * G, embed_dim)
        tokens = {
            'fg': prompts[:, :G] + fg_token,
            
            'bg': value
        }
        x = self.patch_embed(x)  # [B, N ,C]
        cls_token = expand_to_batch(self.cls_token, x.shape[0])  # [B, 1, C]
        if self.dist_token is None:
            x = cat_token(cls_token, x)     # [B, N+1, C]
        else:
            dist_token = expand_to_batch(self.dist_token, x.shape[0])  # [B, 1, C]
            x = cat_token(cls_token, dist_token, x)     # [B, N+2, C]
        x = x + self.pos_embed  # [B, N + 1, C']
        x = self.pos_drop(x)

        S = self.opt.shot
        B = x.shape[0] // (S + 1)

        # Concat image and tokens
        _, N, C = x.shape
        n1 = tokens['fg'].shape[1]
        n2 = tokens['bg'].shape[1]
        fg_token = tokens['fg'].view(B, 1, n1, C).expand(-1, S + 1, -1, -1).reshape(B*(S+1), n1, C)
        bg_token = tokens['bg'].view(B, 1, n2, C).expand(-1, S + 1, -1, -1).reshape(B*(S+1), n2, C)
        x = cat_token(x, fg_token, bg_token)

        # Forward transformer blocks
        for i, block in enumerate(self.blocks):
            x = block(x)
            x = self.reduce_and_expand(x, num=S + 1, start=self.num_tokens, end=n1 + n2)

        # Split outputs
        x, fg_token, bg_token = x.split([N, n1, n2], dim=1)
        tokens['fg'] = fg_token.view(B, S + 1, n1, C)[:, 0]
        tokens['bg'] = bg_token.view(B, S + 1, n2, C)[:, 0]

        x = self.norm(x)

        x = x[:, self.num_tokens:, :]
        b, n, c = x.shape
        hh = int(math.sqrt(n))
        x = x.view(b, hh, hh, c).permute(0, 3, 1, 2).contiguous()

        tokens['fg'] = tokens['fg'].mean(1)[:, :, None, None]    # [B, C, 1, 1]
        if self.opt.bg_num == 1:
            tokens['bg'] = tokens['bg'].mean(1)[:, :, None, None]    # [B, C, 1, 1]
        else:
            _B, _N, _C = tokens['bg'].shape
            bg_num = self.opt.bg_num
            tokens['bg'] = tokens['bg'].reshape(
                _B * bg_num, _N // bg_num, _C).mean(1)[:, :, None, None]    # [Bk, C, 1, 1]

        return dict(out=x, tokens=tokens)

    @staticmethod
    def reduce_and_expand(x, num, start=1, end=0):
        B, N, C = x.shape
        x = x.view(B // num, num, N, C)
        if start > 0:
            mean_token = x[:, :, :start, :].mean(1, keepdim=True).expand(-1, num, -1, -1)
            x[:, :, :start, :] = mean_token
        if end > 0:
            mean_token = x[:, :, -end:, :].mean(1, keepdim=True).expand(-1, num, -1, -1)
            x[:, :, -end:, :] = mean_token
        return x.view(B, N, C)


def expand_to_batch(prompt, batch_size, stack=False):
    if stack:
        prompt = prompt.unsqueeze(0)
    return prompt.expand(batch_size, *[-1 for _ in prompt.shape[1:]])

def cat_token(*args):
    return torch.cat(args, dim=1)

def _init_vit_weights(module: nn.Module, name: str = '', head_bias: float = 0., jax_impl: bool = False):
    """ ViT weight initialization
    * When called without n, head_bias, jax_impl args it will behave exactly the same
      as my original init for compatibility with prev hparam / downstream use cases (ie DeiT).
    * When called w/ valid n (module name) and jax_impl=True, will (hopefully) match JAX impl
    """
    if isinstance(module, nn.Linear):
        if name.startswith('head'):
            nn.init.zeros_(module.weight)
            nn.init.constant_(module.bias, head_bias)
        elif name.startswith('pre_logits'):
            utils.lecun_normal_(module.weight)
            nn.init.zeros_(module.bias)
        else:
            if jax_impl:
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    if 'mlp' in name:
                        nn.init.normal_(module.bias, std=1e-6)
                    else:
                        nn.init.zeros_(module.bias)
            else:
                utils.trunc_normal_(module.weight, std=.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
    elif jax_impl and isinstance(module, nn.Conv2d):
        # NOTE conv was left to pytorch default in my original init
        utils.lecun_normal_(module.weight)
        if module.bias is not None:
            nn.init.zeros_(module.bias)
    elif isinstance(module, (nn.LayerNorm, nn.GroupNorm, nn.BatchNorm2d)):
        nn.init.zeros_(module.bias)
        nn.init.ones_(module.weight)


@torch.no_grad()
def _load_weights_npz(model: VisionTransformer, checkpoint_path: str, prefix: str = ''):
    """ Load weights from .npz checkpoints for official Google Brain Flax implementation
    """
    import numpy as np

    def _n2p(w, t=True):
        if w.ndim == 4 and w.shape[0] == w.shape[1] == w.shape[2] == 1:
            w = w.flatten()
        if t:
            if w.ndim == 4:
                w = w.transpose([3, 2, 0, 1])
            elif w.ndim == 3:
                w = w.transpose([2, 0, 1])
            elif w.ndim == 2:
                w = w.transpose([1, 0])
        return torch.from_numpy(w)

    w = np.load(checkpoint_path)
    if not prefix and 'opt/target/embedding/kernel' in w:
        prefix = 'opt/target/'

    if hasattr(model.patch_embed, 'backbone'):
        # hybrid
        backbone = model.patch_embed.backbone
        stem_only = not hasattr(backbone, 'stem')
        stem = backbone if stem_only else backbone.stem
        stem.conv.weight.copy_(utils.adapt_input_conv(stem.conv.weight.shape[1], _n2p(w[f'{prefix}conv_root/kernel'])))
        stem.norm.weight.copy_(_n2p(w[f'{prefix}gn_root/scale']))
        stem.norm.bias.copy_(_n2p(w[f'{prefix}gn_root/bias']))
        if not stem_only:
            for i, stage in enumerate(backbone.stages):
                for j, block in enumerate(stage.blocks):
                    bp = f'{prefix}block{i + 1}/unit{j + 1}/'
                    for r in range(3):
                        getattr(block, f'conv{r + 1}').weight.copy_(_n2p(w[f'{bp}conv{r + 1}/kernel']))
                        getattr(block, f'norm{r + 1}').weight.copy_(_n2p(w[f'{bp}gn{r + 1}/scale']))
                        getattr(block, f'norm{r + 1}').bias.copy_(_n2p(w[f'{bp}gn{r + 1}/bias']))
                    if block.downsample is not None:
                        block.downsample.conv.weight.copy_(_n2p(w[f'{bp}conv_proj/kernel']))
                        block.downsample.norm.weight.copy_(_n2p(w[f'{bp}gn_proj/scale']))
                        block.downsample.norm.bias.copy_(_n2p(w[f'{bp}gn_proj/bias']))
        embed_conv_w = _n2p(w[f'{prefix}embedding/kernel'])
    else:
        embed_conv_w = utils.adapt_input_conv(
            model.patch_embed.proj.weight.shape[1], _n2p(w[f'{prefix}embedding/kernel']))
    model.patch_embed.proj.weight.copy_(embed_conv_w)
    model.patch_embed.proj.bias.copy_(_n2p(w[f'{prefix}embedding/bias']))
    model.cls_token.copy_(_n2p(w[f'{prefix}cls'], t=False))
    pos_embed_w = _n2p(w[f'{prefix}Transformer/posembed_input/pos_embedding'], t=False)
    if pos_embed_w.shape != model.pos_embed.shape:
        pos_embed_w = resize_pos_embed(  # resize pos embedding when different size from pretrained weights
            pos_embed_w, model.pos_embed, getattr(model, 'num_tokens', 1), model.patch_embed.grid_size)
    model.pos_embed.copy_(pos_embed_w)
    model.norm.weight.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/scale']))
    model.norm.bias.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/bias']))
    if isinstance(model.head, nn.Linear) and model.head.bias.shape[0] == w[f'{prefix}head/bias'].shape[-1]:
        model.head.weight.copy_(_n2p(w[f'{prefix}head/kernel']))
        model.head.bias.copy_(_n2p(w[f'{prefix}head/bias']))
    if isinstance(getattr(model.pre_logits, 'fc', None), nn.Linear) and f'{prefix}pre_logits/bias' in w:
        model.pre_logits.fc.weight.copy_(_n2p(w[f'{prefix}pre_logits/kernel']))
        model.pre_logits.fc.bias.copy_(_n2p(w[f'{prefix}pre_logits/bias']))
    for i, block in enumerate(model.blocks.children()):
        block_prefix = f'{prefix}Transformer/encoderblock_{i}/'
        mha_prefix = block_prefix + 'MultiHeadDotProductAttention_1/'
        block.norm1.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/scale']))
        block.norm1.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/bias']))
        block.attn.qkv.weight.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/kernel'], t=False).flatten(1).T for n in ('query', 'key', 'value')]))
        block.attn.qkv.bias.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/bias'], t=False).reshape(-1) for n in ('query', 'key', 'value')]))
        block.attn.proj.weight.copy_(_n2p(w[f'{mha_prefix}out/kernel']).flatten(1))
        block.attn.proj.bias.copy_(_n2p(w[f'{mha_prefix}out/bias']))
        for r in range(2):
            getattr(block.mlp, f'fc{r + 1}').weight.copy_(_n2p(w[f'{block_prefix}MlpBlock_3/Dense_{r}/kernel']))
            getattr(block.mlp, f'fc{r + 1}').bias.copy_(_n2p(w[f'{block_prefix}MlpBlock_3/Dense_{r}/bias']))
        block.norm2.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_2/scale']))
        block.norm2.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_2/bias']))


@torch.no_grad()
def _load_weights_pth(logger, model: VisionTransformer, checkpoint_path):
    ckpt = torch.load(checkpoint_path, map_location='cpu')['model']
    state_dict = model.state_dict()
    if ckpt['pos_embed'].shape != state_dict['pos_embed'].shape:
        ckpt['pos_embed'] = resize_pos_embed(
            ckpt['pos_embed'], state_dict['pos_embed'], model.num_tokens, model.patch_embed.grid_size)

    counter = 0
    for k in state_dict.keys():
        if k in ckpt:
            state_dict[k] = ckpt[k]
            counter += 1

    logger.info(' ' * 5 + f"==> {counter} parameters loaded.")
    model.load_state_dict(state_dict, strict=True)


def resize_pos_embed(posemb, posemb_new, num_tokens=1, gs_new=()):
    # Rescale the grid of position embeddings when loading from state_dict. Adapted from
    # https://github.com/google-research/vision_transformer/blob/00883dd691c63a6830751563748663526e811cee/vit_jax/checkpoint.py#L224
    _logger.info(' ' * 5 + '==> Resized position embedding: %s to %s', posemb.shape, posemb_new.shape)
    ntok_new = posemb_new.shape[1]
    if num_tokens:
        posemb_tok, posemb_grid = posemb[:, :num_tokens], posemb[0, num_tokens:]
        ntok_new -= num_tokens
    else:
        posemb_tok, posemb_grid = posemb[:, :0], posemb[0]
    gs_old = int(math.sqrt(len(posemb_grid)))
    if not len(gs_new):  # backwards compatibility
        gs_new = [int(math.sqrt(ntok_new))] * 2
    assert len(gs_new) >= 2
    _logger.info(' ' * 5 + '==> Position embedding grid-size from %s to %s', [gs_old, gs_old], gs_new)
    posemb_grid = posemb_grid.reshape(1, gs_old, gs_old, -1).permute(0, 3, 1, 2)
    posemb_grid = F.interpolate(posemb_grid, size=gs_new, mode='bicubic', align_corners=False)
    posemb_grid = posemb_grid.permute(0, 2, 3, 1).reshape(1, gs_new[0] * gs_new[1], -1)
    posemb = torch.cat([posemb_tok, posemb_grid], dim=1)
    return posemb


def checkpoint_filter_fn(state_dict, model):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    if 'model' in state_dict:
        # For deit models
        state_dict = state_dict['model']
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k and len(v.shape) < 4:
            # For old models that I trained prior to conv based patchification
            O, I, H, W = model.patch_embed.proj.weight.shape
            v = v.reshape(O, -1, H, W)
        elif k == 'pos_embed' and v.shape != model.pos_embed.shape:
            # To resize pos embedding when using model at different size from pretrained weights
            v = resize_pos_embed(
                v, model.pos_embed, getattr(model, 'num_tokens', 1), model.patch_embed.grid_size)
        out_dict[k] = v
    return out_dict


vit_factory = {
    'ViT-Ti/16':         {'patch_size': 16, 'embed_dim':  192, 'depth': 12, 'num_heads':  3, 'distilled': False},
    'ViT-S/32':          {'patch_size': 32, 'embed_dim':  384, 'depth': 12, 'num_heads':  6, 'distilled': False},
    'ViT-S/16':          {'patch_size': 16, 'embed_dim':  384, 'depth': 12, 'num_heads':  6, 'distilled': False},
    'ViT-S/16-i21k':     {'patch_size': 16, 'embed_dim':  384, 'depth': 12, 'num_heads':  6, 'distilled': False},
    'ViT-B/32':          {'patch_size': 32, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-B/16':          {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-B/16-384':      {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-B/16-i21k':     {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-B/16-i21k-384': {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-B/8':           {'patch_size':  8, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': False},
    'ViT-L/32':          {'patch_size': 32, 'embed_dim': 1024, 'depth': 24, 'num_heads': 16, 'distilled': False},
    'ViT-L/16':          {'patch_size': 16, 'embed_dim': 1024, 'depth': 24, 'num_heads': 16, 'distilled': False},
    'ViT-L/16-384':      {'patch_size': 16, 'embed_dim': 1024, 'depth': 24, 'num_heads': 16, 'distilled': False},

    'DeiT-T/16':         {'patch_size': 16, 'embed_dim':  192, 'depth': 12, 'num_heads': 3, 'distilled': True},
    'DeiT-S/16':         {'patch_size': 16, 'embed_dim':  384, 'depth': 12, 'num_heads': 6, 'distilled': True},
    'DeiT-B/16':         {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': True},
    'DeiT-B/16-384':     {'patch_size': 16, 'embed_dim':  768, 'depth': 12, 'num_heads': 12, 'distilled': True},
}


def vit_model(model_type,
              image_size,
              pretrained="",
              init_channels=3,
              num_classes=1000,
              opt=None,
              logger=None,
              original=False,
              depth=None):
    return VisionTransformer(img_size=image_size,
                             patch_size=vit_factory[model_type]['patch_size'],
                             in_chans=init_channels,
                             num_classes=num_classes,
                             embed_dim=vit_factory[model_type]['embed_dim'],
                             depth=depth or opt.vit_depth or vit_factory[model_type]['depth'],
                             num_heads=vit_factory[model_type]['num_heads'],
                             pretrained=pretrained,
                             distilled=vit_factory[model_type]['distilled'],
                             opt=opt,
                             logger=logger,
                             original=original)




Overwriting /kaggle/working/FPTrans/networks/vit.py
